In [2]:
#from google.colab import drive
#drive.mount('/drive')

In [3]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input, decode_predictions
import numpy as np
import tensorflow_datasets as tfds

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from tensorflow.python.ops import bitwise_ops
import random
from copy import deepcopy

ds = tfds.load('imagenet_v2', split='test', shuffle_files=False)

Shuffling and writing examples to /root/tensorflow_datasets/imagenet_v2/matched-frequency/1.0.0.incompleteG96DQ6/imagenet_v2-test.tfrecord


Dataset imagenet_v2 downloaded and prepared to /root/tensorflow_datasets/imagenet_v2/matched-frequency/1.0.0. Subsequent calls will reuse this data.


In [4]:
# Add denormal filtering codea
# Add 0-mantissa code

# Custom rounding code to round values inside tensors
# Need to apply fix on nearest and stochastic to the custom_round_not_tensor
# Or merge into 2 function with options
def custom_round(t_value, round_mode = "truncate", skip_processing = False, block_round_mode = "fine_grain_2_contiguous_exponent_reuse",
                 num_exp = 8, num_mantissa = 23, radix_exp = 2, block_size = 128, 
                 radix_mantissa = 2, is_tensor = True):
  '''
    is_tensor will toggle between treating t_value as a tensor vs a non-tensor
    Assume Float 32 - TODO: check data type and round accordingly
    Round all values inside t_value tensor based on the following:
    round_mode: truncate, nearest, stochastic, hw_stochastic, custom
    num_exp: number of bits for exponent
    num_mantissa: number of bits for mantissa
    radix_exp: base number for representation of exponent
    radix_mantissa: base number for representation of mantissa 
  '''
  MAN_MASK = 0x007FFFFF 
  EXP_MASK = 0x7F800000
  SIGN_MASK = 0x80000000
  SIGN_EXP_MASK = 0xFF800000
  GUARD_MASK = 0x00000007
  GUARD_RIGHT = 0x00000001
  GUARD_MID = 0x00000002 
  GUARD_LEFT = 0x00000004

  # This mask is shifted to the right by reduce_num
  # Upper bits ORd with 1's
  TRUNCATE_MASK = 0xFF800000

  # This will point to the digit after the last digit to keep
  NEAREST_PTR = 0x00400000
  LOWEST_PTR = 0x00000001

  MAN_OVERFLOW = 0x00800000

  LARGEST_DENORMAL_EXP_MASK = 0x7F800000
  SMALLEST_DENORMAL_EXP_MASK = 0x00000000

  IMPLICIT_ONE = 0x00800000

  # Number of bits to reduce based on new number of mantissa bits
  reduce_num = 23 - num_mantissa

  # need to traverse the tensorfine_grain
  # convert tensor to numpy array
  if skip_processing == True:
    length = 1
  else:
    length = len(t_value)

  for t_num in range(length):
    if (skip_processing == False):
      if is_tensor:
        x = t_value[t_num].numpy()
      else:
        x = t_value[t_num]
    else:
       x = t_value

    temp_convert_x = tf.bitcast(x, tf.int32)
    temp_exp_bits = bitwise_ops.bitwise_and(temp_convert_x, EXP_MASK)
    temp_exp_bits = bitwise_ops.right_shift(temp_exp_bits,23)
    temp_max = tf.math.reduce_max(temp_exp_bits)
    temp_min = tf.math.reduce_min(temp_exp_bits)
    temp_range = temp_max - temp_min
    #print("local exp range: ", temp_range)

    #print("#############################")
    #print("This is the original tensor: ", x)

    if (round_mode == "truncate"):
      # Code to truncate the mantissa to the new number of bits
      mask = int(TRUNCATE_MASK)
      ##print("This is the original mask: ",hex(mask))
      for j in range(0, num_mantissa):
        mask = mask >> 1
        mask = mask | 0x80000000
      #mask = mask >> reduce_num
      #print("This is the new mask: ",hex(mask))
      # Need to cast float into integer format in order to apply bitwise operations
      # then convert back
      convert_x = tf.bitcast(x, tf.int32)
      convert_x = bitwise_ops.bitwise_and(convert_x, mask)
      x = tf.bitcast(convert_x, tf.float32)

      if is_tensor:
        t_value[t_num] = tf.constant(x)
      else:
        t_value[t_num] = x

      #print("This is the new tensor: ", t_value[t_num].numpy())
      #print("#############################") 
      
    elif (round_mode == "afp_sector_update_jamming"):

      # Use the same exponent per tensor
      # Truncate bits that are shifted off - TODO: Look into rounding

      convert_x = tf.bitcast(x, tf.int32)

      sign_bit = bitwise_ops.bitwise_and(convert_x, SIGN_MASK)
      exp_bits = bitwise_ops.bitwise_and(convert_x, EXP_MASK)
      man_bits = bitwise_ops.bitwise_and(convert_x, MAN_MASK)
      save_man_bits = man_bits

      exp_bits = bitwise_ops.right_shift(exp_bits,23)
      saved_exp = exp_bits

      max_exp = tf.math.reduce_max(exp_bits)
      shifted_max_exp = bitwise_ops.left_shift(max_exp, 23)

      max_tensor = tf.ones_like(exp_bits)
      max_tensor = tf.math.multiply(max_exp, max_tensor)
      diff_tensor = tf.math.subtract(max_tensor,exp_bits)

      ####################
      # Sector Update Code
      # New code to profile intervals
      # print("shifted_man_bits: ", save_man_bits)
      interval_num = []

      for i in range(8):
        interval_num.append(0)

      shifted_man_bits = bitwise_ops.right_shift(save_man_bits, 1)
      shifted_man_bits = bitwise_ops.bitwise_or(shifted_man_bits, NEAREST_PTR)
      shifted_man_bits = bitwise_ops.right_shift(shifted_man_bits, diff_tensor)
      # implicit_one = tf.math.equal(exp_bits, max_exp) 
      # implicit_mask = tf.where(implicit_one, NEAREST_PTR, 0)
      # shifted_man_bits = bitwise_ops.bitwise_or(shifted_man_bits, implicit_mask)
      truncate_mask2 = NEAREST_PTR 
      # for i in range(num_mantissa - 1):
      for i in range(2):
        truncate_mask2 = truncate_mask2 >> 1 | NEAREST_PTR

      #print("truncate_mask2: ", truncate_mask2)
      #print("shifted_man_bits: ", shifted_man_bits)
      interval_bits = bitwise_ops.bitwise_and(shifted_man_bits, truncate_mask2)
      #print("masked shift_man interval_bits:")
      #print(interval_bits)
      interval_bits = bitwise_ops.right_shift(interval_bits, 20)
      interval_int = tf.bitcast(interval_bits, tf.int32)
      #print("interval_bits:")
      #print(interval_bits)
      #print("interval_int: ", interval_bits)

      one_hot_interval = tf.one_hot(interval_int, 8)
      #print("one_hot_interval:")
      #print(one_hot_interval)

      for i in range(8):
        for j in range(len(one_hot_interval)):
          interval_num[i] += one_hot_interval[j].numpy()

      #####
      interval_bool = deepcopy(interval_num[0])
      #print("interval_count: ", interval_bool)
      #interval_bool = deepcopy(interval_num)
      free_encodings = 0
      for i in range(8):
        if interval_bool[i] > 0:
          interval_bool[i] = 1
        else:
          free_encodings += 1
      #print("Free Encodings: ", free_encodings)

      ##############
      # All same sign update
      min_value = tf.math.reduce_min(x)
      max_value = tf.math.reduce_max(x)
      all_positive = min_value >= 0
      all_negative = max_value <= 0
      all_one_sign = all_positive or all_negative

      ##################
      # range code
      abs_x = tf.math.abs(x)
      max_abs_value = tf.math.reduce_max(abs_x)
      min_abs_value = tf.math.reduce_min(abs_x)

      # Code to check if min value in block is > 0001 assuming 3-bit mantissa
      min_over_half = max_exp - 3
      min_over_half = bitwise_ops.left_shift(min_over_half, 23)
      min_over_half = tf.bitcast(min_over_half, tf.float32)
      min_just_over_half = min_abs_value > min_over_half
      # Code to check for round down to 2^max_exp
      
      range_mask1 = int(NEAREST_PTR) >> 2
      # use num_mantissa - 2 to find LSB, use num_mantissa - 1 to find bit to right of LSB to check for rounding down
      #for j in range(0, num_mantissa-1):
      #  range_mask1 = range_mask1 >> 1
      
      # truncate max_exp - instead of checking for 1001 (for 3 bit mantissa)
      # check for 100
      # over_half = shifted_max_exp # bitwise_ops.left_shift(max_exp, 23)

      over_half = bitwise_ops.bitwise_or(shifted_max_exp, range_mask1)

      just_over_half = tf.bitcast(over_half, tf.float32)

      high_exp = max_exp
      if (high_exp != 0):
        high_exp -= 127

      round_down_to_half = max_abs_value < just_over_half
      round_down_to_one_inc = False
      round_down_to_two_inc = False

      extra_num_exp = 0
      max_value_float = 0.0
      max_mantissa = 0

      # If there are gaps in the double precision
      # high threshold describes the upper bound for double precision
      high_threshold = 0.0
      low_threshold = 0.0
      min_mantissa = 0
      high_precision_gap = False

      if (all_one_sign == True):
        # num_mantissa += 1
        free_encodings += 8

      original_num = num_mantissa
      #print("New block - remaining free encodings: ", free_encodings)
      # Code to check for 2x precision - equivalent to 1 extra bit
      if (free_encodings > 0):
        if all_one_sign == False:
          required_encodings = 8 - free_encodings
        else:
          required_encodings = 16 - free_encodings

        while (free_encodings >= required_encodings):
          num_mantissa += 1
          free_encodings -= required_encodings
          required_encodings = 2*required_encodings

      #print("num_mantissa: (original), (new) ", original_num, num_mantissa)
      #print("remaining free encodings: ", free_encodings)
      # check for remaining free encodings

      if (free_encodings > 0):
        int_num = 0
        # Need to add code here to selectively increase accuracy for sectors
        # interval_bool = 2 for sections with higher accuracy
        while free_encodings > 0:
          if interval_bool[int_num] >= 1:
            interval_bool[int_num] += 1
            free_encodings -= 1
          int_num = (int_num + 1) % 8

      ############################
      # Create tensor with flags specific to each value to indicate increase in accuracy
      # Flag to indicate select range for higher precision
      # high_accuracy_sector stores the number of extra bits of precision for the sector
      high_accuracy_sector = tf.zeros_like(convert_x)
      for i in range(8):
        temp_bool = tf.math.equal(interval_int, i)
        high_accuracy_sector = tf.where(temp_bool, interval_bool[i] -1 , high_accuracy_sector)

      negative_sector = tf.math.less(high_accuracy_sector, 0)
      high_accuracy_sector = tf.where(negative_sector, 0, high_accuracy_sector)
      #print("high_accuracy_sector: ", high_accuracy_sector)
      ############################

      #print("after while loop")
      if (interval_bool[0] > 0):
        zero_threshold = num_mantissa - 1 + interval_bool[0] - 1
      else:
        zero_threshold = num_mantissa - 1  
      shift_to_zero = tf.math.greater(diff_tensor, zero_threshold) 

      # Based on the difference of the exponent, round the bits shifted - start with truncation
      exp_bits = bitwise_ops.left_shift(exp_bits,23)
      man_bits = bitwise_ops.right_shift(man_bits, diff_tensor)

      # Need to mask out the bits lower than the num_mantissa bits
      mask = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa-1):
        mask = mask >> 1
        mask = mask | 0x80000000

      mask1 = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa):
        mask1 = mask1 >> 1
        mask1 = mask1 | 0x80000000

      mask2 = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa + 1):
        mask2 = mask2 >> 1
        mask2 = mask2 | 0x80000000
      
      mask3 = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa + 2):
        mask3 = mask3 >> 1
        mask3 = mask3 | 0x80000000      
      ###########################
      # select high precision code
      tensor_mask = tf.ones_like(convert_x)
      tensor_mask = tf.math.multiply(tensor_mask, mask)
      equal_1 = tf.equal(high_accuracy_sector, 1)
      tensor_mask = tf.where(equal_1, mask1, tensor_mask)
      equal_2 = tf.equal(high_accuracy_sector, 2)
      tensor_mask = tf.where(equal_2, mask2, tensor_mask)
      equal_3 = tf.equal(high_accuracy_sector, 3)
      tensor_mask = tf.where(equal_3, mask3, tensor_mask)   
      #print("mask, 1, 2, 3: ", mask, mask1, mask2, mask3)
      #print("equal_1, 2, 3: ", equal_1, equal_2, equal_3)
      #print("tensor_mask: ", tensor_mask)

      ######### Round Code ###########
      ## Adding jamming code
      ## setting mask with 3 on bits
      guard_mask_save = NEAREST_PTR
      for i in range(2):
        guard_mask_save = guard_mask_save >> 1 |NEAREST_PTR
        
      for j in range(num_mantissa - 1):
        guard_mask = guard_mask_save >> 1

      guard_mask1 = guard_mask_save
      for j in range(num_mantissa):
        guard_mask1 = guard_mask1 >> 1

      guard_mask2 = guard_mask_save
      for j in range(num_mantissa + 1):
        guard_mask2 = guard_mask2 >> 1

      guard_mask3 = guard_mask_save
      for j in range(num_mantissa + 2):
        guard_mask3 = guard_mask3 >> 1
      ###########################
      # select high precision code
      tensor_guard_mask = tf.ones_like(convert_x)
      tensor_guard_mask = tf.math.multiply(tensor_mask, guard_mask)
      #equal_1 = tf.equal(high_accuracy_sector, 1)
      tensor_guard_mask = tf.where(equal_1, guard_mask1, tensor_guard_mask)
      #equal_2 = tf.equal(high_accuracy_sector, 2)
      tensor_guard_mask = tf.where(equal_2, guard_mask2, tensor_guard_mask)
      #equal_3 = tf.equal(high_accuracy_sector, 3)
      tensor_guard_mask = tf.where(equal_3, guard_mask3, tensor_guard_mask)  

      #print("tensor_guard_mask: ", tensor_guard_mask)

      ##########################
      # Create a 1 in the lsb of each value (lsb_mask)
      # Use this as a mask to get the actual lsb,
      # zero out this mask based on the condition (actual lsb is 0 and guard bits are all 0)

      lsb_mask = NEAREST_PTR << 1
      for j in range(num_mantissa - 1):
        lsb_mask = lsb_mask >> 1

      lsb_mask1 = NEAREST_PTR << 1
      for j in range(num_mantissa):
        lsb_mask1 = lsb_mask1 >> 1

      lsb_mask2 = NEAREST_PTR << 1
      for j in range(num_mantissa + 1):
        lsb_mask2 = lsb_mask2 >> 1

      lsb_mask3 = NEAREST_PTR << 1
      for j in range(num_mantissa + 2):
        lsb_mask3 = lsb_mask3 >> 1

      tensor_lsb_mask = tf.ones_like(convert_x)
      tensor_lsb_mask = tf.math.multiply(tensor_lsb_mask, lsb_mask)
      #equal_1 = tf.equal(high_accuracy_sector, 1)
      tensor_lsb_mask = tf.where(equal_1, lsb_mask1, tensor_lsb_mask)
      #equal_2 = tf.equal(high_accuracy_sector, 2)
      tensor_lsb_mask = tf.where(equal_2, lsb_mask2, tensor_lsb_mask)
      #equal_3 = tf.equal(high_accuracy_sector, 3)
      tensor_lsb_mask = tf.where(equal_3, lsb_mask3, tensor_lsb_mask)  

      #print("tensor_lsb_mask: ", tensor_lsb_mask)
      tensor_lsb_bit = bitwise_ops.bitwise_and(man_bits, tensor_lsb_mask)
      lsb_bit = tf.cast(tensor_lsb_bit, tf.bool)
      #print("lsb_bit: ", lsb_bit)
      
      #print("guard_mask ", guard_mask)
      #print("tensor_guard_mask: ", tensor_guard_mask)
      ######### Round Code ###########

      #print("guard_mask: ", guard_mask)
      # guard_bit = bitwise_ops.bitwise_and(man_bits, guard_mask)
      #print("man_bits: ", man_bits)
      guard_bit = bitwise_ops.bitwise_and(man_bits, tensor_guard_mask)
      #print("3 guard_bits: ", guard_bit)
      ######### Jamming Code
      # guard_bit is true if any of guard bits is a 1 -  jamming
      guard_bit = tf.cast(guard_bit, tf.bool)
      jamming_lsb = tf.logical_or(lsb_bit, guard_bit)
      jamming_mantissa = bitwise_ops.bitwise_or(man_bits, tensor_lsb_mask)
      #print("jamming_lsb, mantissa: ", jamming_lsb, jamming_mantissa)
      man_bits = tf.where(jamming_lsb, jamming_mantissa, man_bits)

      not_shift_zero = tf.math.less_equal(diff_tensor, zero_threshold)
      guard_bit = tf.logical_and(guard_bit, not_shift_zero)

      #print("guard_bit: ", guard_bit)

      #### Need to work on this code to enable rounding!!!
      # final_round_exp = tf.math.subtract(saved_exp, num_mantissa - 1)
      '''
      final_round_exp = tf.math.subtract(saved_exp, num_mantissa - 1)
      final_round_exp = tf.math.subtract(final_round_exp, high_accuracy_sector)
      final_round_exp = tf.math.add(final_round_exp, diff_tensor) 
      final_round_exp = bitwise_ops.left_shift(final_round_exp,23)
      final_round_value = bitwise_ops.bitwise_or(sign_bit, final_round_exp)
      final_round_float = tf.bitcast(final_round_value, tf.float32)
      '''
      #print("final_round_float: ", final_round_float)
      
      ###############################

      # man_bits = bitwise_ops.bitwise_and(man_bits, mask)
      man_bits = bitwise_ops.bitwise_and(man_bits, tensor_mask)
      man_bits = bitwise_ops.left_shift(man_bits, diff_tensor)

      final_value = bitwise_ops.bitwise_or(sign_bit,exp_bits)
      final_value = bitwise_ops.bitwise_or(final_value, man_bits)

      zeros_tensor = tf.zeros_like(man_bits)
      
      exp_only = tf.math.greater(diff_tensor, num_mantissa - 1)
      not_shift_zero = tf.math.less_equal(diff_tensor, zero_threshold)
      exp_only = tf.logical_and(exp_only, not_shift_zero)
      sign_exp_bits = bitwise_ops.bitwise_and(convert_x, SIGN_EXP_MASK)
      zeros_tensor = tf.where(exp_only, sign_exp_bits , zeros_tensor  )

      final_value = tf.where(shift_to_zero, zeros_tensor, final_value)

      #final_value = bitwise_ops.bitwise_and(final_value, mask)

      x = tf.bitcast(final_value, tf.float32)

      # x = tf.where(guard_bit, x + final_round_float, x )

      if skip_processing == False:
        if is_tensor:
          t_value[t_num] = tf.constant(x)
        else:
          t_value[t_num] = x
      else:
        t_value = x

      
    elif (round_mode == "afp_sector_update_jamming_zero_new"):

      # Use the same exponent per tensor
      # Truncate bits that are shifted off - TODO: Look into rounding

      convert_x = tf.bitcast(x, tf.int32)

      sign_bit = bitwise_ops.bitwise_and(convert_x, SIGN_MASK)
      exp_bits = bitwise_ops.bitwise_and(convert_x, EXP_MASK)
      man_bits = bitwise_ops.bitwise_and(convert_x, MAN_MASK)
      save_man_bits = man_bits

      exp_bits = bitwise_ops.right_shift(exp_bits,23)
      saved_exp = exp_bits

      max_exp = tf.math.reduce_max(exp_bits)
      shifted_max_exp = bitwise_ops.left_shift(max_exp, 23)

      max_tensor = tf.ones_like(exp_bits)
      max_tensor = tf.math.multiply(max_exp, max_tensor)
      #print("(check) max_tensor: ", max_tensor)
      diff_tensor = tf.math.subtract(max_tensor,exp_bits)

      ####################
      # Sector Update Code
      # New code to profile intervals
      # print("shifted_man_bits: ", save_man_bits)
      interval_num = []

      for i in range(2**num_mantissa):
        interval_num.append(0)

      shifted_man_bits = bitwise_ops.right_shift(save_man_bits, 1)
      shifted_man_bits = bitwise_ops.bitwise_or(shifted_man_bits, NEAREST_PTR)
      shifted_man_bits = bitwise_ops.right_shift(shifted_man_bits, diff_tensor)
      # implicit_one = tf.math.equal(exp_bits, max_exp) 
      # implicit_mask = tf.where(implicit_one, NEAREST_PTR, 0)
      # shifted_man_bits = bitwise_ops.bitwise_or(shifted_man_bits, implicit_mask)
      truncate_mask2 = NEAREST_PTR 
      # for i in range(num_mantissa - 1):
      for i in range(num_mantissa -1):
        truncate_mask2 = truncate_mask2 >> 1 | NEAREST_PTR

      # Stopped here - adding as many intervals as quantization levels

      #print("truncate_mask2: ", truncate_mask2)
      #print("shifted_man_bits: ", shifted_man_bits)
      interval_bits = bitwise_ops.bitwise_and(shifted_man_bits, truncate_mask2)
      #print("masked shift_man interval_bits:")
      #print(interval_bits)
      interval_bits = bitwise_ops.right_shift(interval_bits, 23 - num_mantissa)
      interval_int = tf.bitcast(interval_bits, tf.int32)
      #print("interval_bits:")
      #print(interval_bits)
      #print("interval_int: ", interval_bits)
      #print("x: ", x)
      one_hot_interval = tf.one_hot(interval_int, 2**num_mantissa)
      #print("one_hot_interval:")
      #print(one_hot_interval)

      for i in range(2**num_mantissa):
        for j in range(len(one_hot_interval)):
          interval_num[i] += one_hot_interval[j].numpy()

      #####
      interval_bool = deepcopy(interval_num[0])
      interval_num_saved = deepcopy(interval_num[0])
      #print("interval_count: ", interval_bool)
      #interval_bool = deepcopy(interval_num)
      if (len(interval_bool) == 1):
        interval_bool = interval_bool[0]
      #print("interval_bool: ", interval_bool)
      free_encodings = 0
      for i in range(2**num_mantissa):
        if interval_bool[i] > 0:
          interval_bool[i] = 1
        else:
          free_encodings += 1
      #print("Free Encodings: ", free_encodings)

      ##############
      # All same sign update
      min_value = tf.math.reduce_min(x)
      max_value = tf.math.reduce_max(x)
      all_positive = min_value >= 0
      all_negative = max_value <= 0
      all_one_sign = all_positive or all_negative

      ##################
      # range code
      abs_x = tf.math.abs(x)
      max_abs_value = tf.math.reduce_max(abs_x)
      min_abs_value = tf.math.reduce_min(abs_x)

      # Code to check if min value in block is > 0001 assuming 3-bit mantissa
      min_over_half = max_exp - 3
      min_over_half = bitwise_ops.left_shift(min_over_half, 23)
      min_over_half = tf.bitcast(min_over_half, tf.float32)
      min_just_over_half = min_abs_value > min_over_half
      # Code to check for round down to 2^max_exp
      
      range_mask1 = int(NEAREST_PTR) >> 2
      # use num_mantissa - 2 to find LSB, use num_mantissa - 1 to find bit to right of LSB to check for rounding down
      #for j in range(0, num_mantissa-1):
      #  range_mask1 = range_mask1 >> 1
      
      # truncate max_exp - instead of checking for 1001 (for 3 bit mantissa)
      # check for 100
      # over_half = shifted_max_exp # bitwise_ops.left_shift(max_exp, 23)

      over_half = bitwise_ops.bitwise_or(shifted_max_exp, range_mask1)

      just_over_half = tf.bitcast(over_half, tf.float32)

      high_exp = max_exp
      if (high_exp != 0):
        high_exp -= 127

      round_down_to_half = max_abs_value < just_over_half
      round_down_to_one_inc = False
      round_down_to_two_inc = False

      extra_num_exp = 0
      max_value_float = 0.0
      max_mantissa = 0

      # If there are gaps in the double precision
      # high threshold describes the upper bound for double precision
      high_threshold = 0.0
      low_threshold = 0.0
      min_mantissa = 0
      high_precision_gap = False

      if (all_one_sign == True):
        # num_mantissa += 1
        free_encodings += 2**num_mantissa

      original_num = num_mantissa
      #print("New block - remaining free encodings: ", free_encodings)
      # Code to check for 2x precision - equivalent to 1 extra bit
      if (free_encodings > 0):
        if all_one_sign == False:
          required_encodings = (2**num_mantissa) - free_encodings
        else:
          required_encodings = 2*(2**num_mantissa) - free_encodings

        while (free_encodings >= required_encodings):
          num_mantissa += 1
          free_encodings -= required_encodings
          required_encodings = 2*required_encodings

      #print("num_mantissa: (original), (new), free encodings  ", original_num, num_mantissa, free_encodings)
      #if (num_mantissa - original_num > 3):
      #  print("x: ", x)
      #  print("interval_bool: ", interval_bool)
      #print("remaining free encodings: ", free_encodings)
      # check for remaining free encodings

      if (free_encodings > 0):
        int_num = 0
        # Need to add code here to selectively increase accuracy for sectors
        # interval_bool = 2 for sections with higher accuracy
        while free_encodings > 0:
          if interval_bool[int_num] >= 1:
            interval_bool[int_num] += 1
            free_encodings -= 1
          int_num = (int_num + 1) % (2**original_num)

      ############################
      # Create tensor with flags specific to each value to indicate increase in accuracy
      # Flag to indicate select range for higher precision
      # high_accuracy_sector stores the number of extra bits of precision for the sector
      high_accuracy_sector = tf.zeros_like(convert_x)
      for i in range(2**original_num):
        temp_bool = tf.math.equal(interval_int, i)
        high_accuracy_sector = tf.where(temp_bool, interval_bool[i] -1 , high_accuracy_sector)

      negative_sector = tf.math.less(high_accuracy_sector, 0)
      high_accuracy_sector = tf.where(negative_sector, 0, high_accuracy_sector)
      #print("interval_bool: ", interval_bool)
      #print("high_accuracy_sector: ", high_accuracy_sector)
      ############################


      # 8_8_21 - Stopped here - for increased intervals change !!!

      #print("after while loop")
      if (interval_bool[0] > 0):
        zero_threshold = num_mantissa - 1 + interval_bool[0] - 1
      else:
        zero_threshold = num_mantissa - 1  
      shift_to_zero = tf.math.greater(diff_tensor, zero_threshold)

      #non_zero_mantissa = tf.math.not_equal(x, 0.0) 
      non_zero_mantissa = tf.math.less_equal(diff_tensor, zero_threshold + 3)
      non_zero_mantissa = tf.math.logical_and(shift_to_zero, non_zero_mantissa)

      # Based on the difference of the exponent, round the bits shifted - start with truncation
      exp_bits = bitwise_ops.left_shift(exp_bits,23)
      man_bits = bitwise_ops.right_shift(man_bits, diff_tensor)

      # Need to mask out the bits lower than the num_mantissa bits
      mask = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa-1):
        mask = mask >> 1
        mask = mask | 0x80000000

      mask1 = (mask >> 1) | 0x80000000
      mask2 = (mask1 >> 1) | 0x80000000      
      mask3 = (mask2 >> 1) | 0x80000000
      mask4 = (mask3 >> 1) | 0x80000000 
      mask5 = (mask4 >> 1) | 0x80000000
      mask6 = (mask5 >> 1) | 0x80000000       
      '''
      mask1 = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa):
        mask1 = mask1 >> 1
        mask1 = mask1 | 0x80000000

      mask2 = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa + 1):
        mask2 = mask2 >> 1
        mask2 = mask2 | 0x80000000
      
      mask3 = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa + 2):
        mask3 = mask3 >> 1
        mask3 = mask3 | 0x80000000

      mask4 = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa + 3):
        mask4 = mask4 >> 1
        mask4 = mask4 | 0x80000000

      mask5 = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa + 4):
        mask5 = mask5 >> 1
        mask5 = mask5 | 0x80000000
      
      mask6 = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa + 5):
        mask6 = mask6 >> 1
        mask6 = mask6 | 0x80000000  
      '''

      ###########################
      # select high precision code
      tensor_mask = tf.ones_like(convert_x)
      tensor_mask = tf.math.multiply(tensor_mask, mask)
      equal_1 = tf.equal(high_accuracy_sector, 1)
      equal_2 = tf.equal(high_accuracy_sector, 2)
      equal_3 = tf.equal(high_accuracy_sector, 3)
      equal_4 = tf.equal(high_accuracy_sector, 4)
      equal_5 = tf.equal(high_accuracy_sector, 5)
      equal_6 = tf.equal(high_accuracy_sector, 6)

      baseline_tensor = bitwise_ops.bitwise_and(man_bits, tensor_mask)

      temp_mask = tf.ones_like(convert_x)
      temp_mask = tf.math.multiply(temp_mask, mask1)      
      temp_tensor = bitwise_ops.bitwise_and(man_bits, temp_mask)
      # Found 1 in higher bit
      overwrite = tf.math.not_equal(baseline_tensor, temp_tensor) 
      overwrite_check = tf.math.greater(high_accuracy_sector, 1)
      overwrite1 = tf.math.logical_and(overwrite, overwrite_check)

      temp_mask = tf.ones_like(convert_x)
      temp_mask = tf.math.multiply(temp_mask, mask2)      
      temp_tensor2 = bitwise_ops.bitwise_and(man_bits, temp_mask)
      # Found 1 in higher bit
      overwrite = tf.math.not_equal(temp_tensor, temp_tensor2) 
      overwrite_check = tf.math.greater(high_accuracy_sector, 3)
      overwrite2 = tf.math.logical_and(overwrite, overwrite_check)
      
      temp_mask = tf.ones_like(convert_x)
      temp_mask = tf.math.multiply(temp_mask, mask3)      
      temp_tensor3 = bitwise_ops.bitwise_and(man_bits, temp_mask)
      # Found 1 in higher bit
      overwrite = tf.math.not_equal(temp_tensor2, temp_tensor3) 
      overwrite_check = tf.math.greater(high_accuracy_sector, 4)
      overwrite3 = tf.math.logical_and(overwrite, overwrite_check)

      temp_mask = tf.ones_like(convert_x)
      temp_mask = tf.math.multiply(temp_mask, mask4)      
      temp_tensor4 = bitwise_ops.bitwise_and(man_bits, temp_mask)
      # Found 1 in higher bit
      overwrite = tf.math.not_equal(temp_tensor3, temp_tensor4) 
      overwrite_check = tf.math.greater(high_accuracy_sector, 5)
      overwrite4 = tf.math.logical_and(overwrite, overwrite_check)

      tensor_mask = tf.where(equal_1, mask1, tensor_mask)
      tensor_mask = tf.where(equal_2, mask2, tensor_mask)
      tensor_mask = tf.where(equal_3, mask3, tensor_mask)
      tensor_mask = tf.where(equal_4, mask1, tensor_mask)
      tensor_mask = tf.where(equal_5, mask2, tensor_mask)
      tensor_mask = tf.where(equal_6, mask3, tensor_mask)     
      #print("mask, 1, 2, 3: ", mask, mask1, mask2, mask3)
      #print("equal_1, 2, 3: ", equal_1, equal_2, equal_3)
      #print("tensor_mask: ", tensor_mask)

      tensor_mask = tf.where(overwrite1, mask1, tensor_mask)
      tensor_mask = tf.where(overwrite2, mask2, tensor_mask)
      tensor_mask = tf.where(overwrite3, mask3, tensor_mask)
      tensor_mask = tf.where(overwrite4, mask4, tensor_mask)
          
      '''
      power_of_two_mask1 = NEAREST_PTR
      power_of_two_mask2 = power_of_two_mask1 >> 1 | NEAREST_PTR
      power_of_two_mask3 = power_of_two_mask2 >> 1 | NEAREST_PTR
      power_of_two_mask4 = power_of_two_mask3 >> 1 | NEAREST_PTR
      power_of_two_mask5 = power_of_two_mask4 >> 1 | NEAREST_PTR
      power_of_two_mask6 = power_of_two_mask5 >> 1 | NEAREST_PTR      

      tensor_power_mask = tf.zeros_like(convert_x)
      tensor_power_mask = tf.where(equal_1, power_of_two_mask1, tensor_power_mask)
      tensor_power_mask = tf.where(equal_2, power_of_two_mask2, tensor_power_mask)
      tensor_power_mask = tf.where(equal_3, power_of_two_mask3, tensor_power_mask)
      tensor_power_mask = tf.where(equal_4, power_of_two_mask4, tensor_power_mask)
      tensor_power_mask = tf.where(equal_5, power_of_two_mask5, tensor_power_mask)
      tensor_power_mask = tf.where(equal_6, power_of_two_mask6, tensor_power_mask)

      tensor_power_mask = bitwise_ops.right_shift(tensor_power_mask, num_mantissa -1)
      '''


      ######### Round Code ###########
      ## Adding jamming code
      ## setting mask with 3 on bits
      guard_mask_save = NEAREST_PTR
      for i in range(2):
        guard_mask_save = (guard_mask_save >> 1) |NEAREST_PTR

      #print("guard_mask_save: ", guard_mask_save)  
      #for j in range(num_mantissa - 1):
      #  guard_mask = guard_mask_save >> 1
      guard_mask = guard_mask_save >> (num_mantissa -1)
      guard_mask1 = guard_mask_save >> num_mantissa
      guard_mask2 = guard_mask_save >> (num_mantissa + 1)
      guard_mask3 = guard_mask_save >> (num_mantissa + 2)
      guard_mask4 = guard_mask_save >> (num_mantissa + 3)
      guard_mask5 = guard_mask_save >> (num_mantissa + 4)
      guard_mask6 = guard_mask_save >> (num_mantissa + 5)    

      # print("guard_mask: ", guard_mask)  
      # print("guard_mask1: ", guard_mask1)  
      # print("guard_mask2: ", guard_mask2)  
      # print("guard_mask3: ", guard_mask3) 
      # print("guard_mask4: ", guard_mask4)  
      # print("guard_mask5: ", guard_mask5) 
      # print("guard_mask6: ", guard_mask6)  

                        
      '''
      for j in range(num_mantissa):
        guard_mask1 = guard_mask1 >> 1

      guard_mask2 = guard_mask_save
      for j in range(num_mantissa + 1):
        guard_mask2 = guard_mask2 >> 1

      guard_mask3 = guard_mask_save
      for j in range(num_mantissa + 2):
        guard_mask3 = guard_mask3 >> 1

      guard_mask4 = guard_mask_save
      for j in range(num_mantissa + 3):
        guard_mask4 = guard_mask4 >> 1

      guard_mask5 = guard_mask_save
      for j in range(num_mantissa + 4):
        guard_mask5 = guard_mask5 >> 1

      guard_mask6 = guard_mask_save
      for j in range(num_mantissa + 5):
        guard_mask6 = guard_mask6 >> 1
      '''
      ###########################
      # select high precision code
      tensor_guard_mask = tf.ones_like(convert_x)
      #tensor_guard_mask = tf.math.multiply(tensor_mask, guard_mask)
      tensor_guard_mask = tf.where(True, guard_mask, guard_mask)
      #print("tensor_guard_mask: ", tensor_guard_mask)
      #equal_1 = tf.equal(high_accuracy_sector, 1)
      tensor_guard_mask = tf.where(equal_1, guard_mask1, tensor_guard_mask)
      #equal_2 = tf.equal(high_accuracy_sector, 2)
      tensor_guard_mask = tf.where(equal_2, guard_mask2, tensor_guard_mask)
      #equal_3 = tf.equal(high_accuracy_sector, 3)
      tensor_guard_mask = tf.where(equal_3, guard_mask3, tensor_guard_mask)  
      tensor_guard_mask = tf.where(equal_4, guard_mask4, tensor_guard_mask)
      tensor_guard_mask = tf.where(equal_5, guard_mask5, tensor_guard_mask)
      tensor_guard_mask = tf.where(equal_6, guard_mask6, tensor_guard_mask)  

      #print("tensor_guard_mask: ", tensor_guard_mask)

      ##########################
      # Create a 1 in the lsb of each value (lsb_mask)
      # Use this as a mask to get the actual lsb,
      # zero out this mask based on the condition (actual lsb is 0 and guard bits are all 0)

      lsb_mask = NEAREST_PTR << 1
      lsb_mask = lsb_mask >> (num_mantissa -1)
      lsb_mask1 = lsb_mask >> 1
      lsb_mask2 = lsb_mask1 >> 1
      lsb_mask3 = lsb_mask2 >> 1
      lsb_mask4 = lsb_mask3 >> 1
      lsb_mask5 = lsb_mask4 >> 1 
      lsb_mask6 = lsb_mask5 >> 1                       
      '''
      for j in range(num_mantissa - 1):
        lsb_mask = lsb_mask >> 1

      lsb_mask1 = NEAREST_PTR << 1
      for j in range(num_mantissa):
        lsb_mask1 = lsb_mask1 >> 1

      lsb_mask2 = NEAREST_PTR << 1
      for j in range(num_mantissa + 1):
        lsb_mask2 = lsb_mask2 >> 1

      lsb_mask3 = NEAREST_PTR << 1
      for j in range(num_mantissa + 2):
        lsb_mask3 = lsb_mask3 >> 1

      lsb_mask4 = NEAREST_PTR << 1
      for j in range(num_mantissa + 3):
        lsb_mask4 = lsb_mask4 >> 1

      lsb_mask5 = NEAREST_PTR << 1
      for j in range(num_mantissa + 4):
        lsb_mask5 = lsb_mask5 >> 1

      lsb_mask6 = NEAREST_PTR << 1
      for j in range(num_mantissa + 5):
        lsb_mask6 = lsb_mask6 >> 1
      '''

      tensor_lsb_mask = tf.ones_like(convert_x)
      tensor_lsb_mask = tf.math.multiply(tensor_lsb_mask, lsb_mask)
      #equal_1 = tf.equal(high_accuracy_sector, 1)
      tensor_lsb_mask = tf.where(equal_1, lsb_mask1, tensor_lsb_mask)
      #equal_2 = tf.equal(high_accuracy_sector, 2)
      tensor_lsb_mask = tf.where(equal_2, lsb_mask2, tensor_lsb_mask)
      #equal_3 = tf.equal(high_accuracy_sector, 3)
      tensor_lsb_mask = tf.where(equal_3, lsb_mask3, tensor_lsb_mask)  
      tensor_lsb_mask = tf.where(equal_4, lsb_mask4, tensor_lsb_mask)
      tensor_lsb_mask = tf.where(equal_5, lsb_mask5, tensor_lsb_mask)
      tensor_lsb_mask = tf.where(equal_6, lsb_mask6, tensor_lsb_mask)  

      #print("tensor_lsb_mask: ", tensor_lsb_mask)
      tensor_lsb_bit = bitwise_ops.bitwise_and(man_bits, tensor_lsb_mask)
      lsb_bit = tf.cast(tensor_lsb_bit, tf.bool)
      #print("lsb_bit: ", lsb_bit)
      
      #print("guard_mask ", guard_mask)
      #print("tensor_guard_mask: ", tensor_guard_mask)
      ######### Round Code ###########

      #print("guard_mask: ", guard_mask)
      # guard_bit = bitwise_ops.bitwise_and(man_bits, guard_mask)
      #print("man_bits: ", man_bits)
      guard_bit = bitwise_ops.bitwise_and(man_bits, tensor_guard_mask)
      #print("3 guard_bits: ", guard_bit)
      ######### Jamming Code
      # guard_bit is true if any of guard bits is a 1 -  jamming
      guard_bit = tf.cast(guard_bit, tf.bool)
      jamming_lsb = tf.logical_or(lsb_bit, guard_bit)
      jamming_mantissa = bitwise_ops.bitwise_or(man_bits, tensor_lsb_mask)
      #print("jamming_lsb, mantissa: ", jamming_lsb, jamming_mantissa)
      man_bits = tf.where(jamming_lsb, jamming_mantissa, man_bits)
      #print("man_bits: ", man_bits)


      not_shift_zero = tf.math.less_equal(diff_tensor, zero_threshold)
      guard_bit = tf.logical_and(guard_bit, not_shift_zero)

      #print("guard_bit: ", guard_bit)

      #### Need to work on this code to enable rounding!!!
      # final_round_exp = tf.math.subtract(saved_exp, num_mantissa - 1)
      '''
      final_round_exp = tf.math.subtract(saved_exp, num_mantissa - 1)
      final_round_exp = tf.math.subtract(final_round_exp, high_accuracy_sector)
      final_round_exp = tf.math.add(final_round_exp, diff_tensor) 
      final_round_exp = bitwise_ops.left_shift(final_round_exp,23)
      final_round_value = bitwise_ops.bitwise_or(sign_bit, final_round_exp)
      final_round_float = tf.bitcast(final_round_value, tf.float32)
      '''
      #print("final_round_float: ", final_round_float)
      
      ###############################

      # man_bits = bitwise_ops.bitwise_and(man_bits, mask)
      man_bits = bitwise_ops.bitwise_and(man_bits, tensor_mask)
      man_bits = bitwise_ops.left_shift(man_bits, diff_tensor)

      final_value = bitwise_ops.bitwise_or(sign_bit,exp_bits)
      final_value = bitwise_ops.bitwise_or(final_value, man_bits)

      zeros_tensor = tf.zeros_like(man_bits)
      
      exp_only = tf.math.greater(diff_tensor, num_mantissa - 1)
      not_shift_zero = tf.math.less_equal(diff_tensor, zero_threshold)
      exp_only = tf.logical_and(exp_only, not_shift_zero)

      # non zero mantissa code
      #exp_only_new = tf.math.logical_and(shift_to_zero, non_zero_mantissa)

      sign_exp_bits = bitwise_ops.bitwise_and(convert_x, SIGN_EXP_MASK)

      # stochastic exp only value code
      # exp = tf.random.uniform() - min = min exp, max is the exp of max - mantissa - extra encodings
      #sign_bit = bitwise_ops.bitwise_and(convert_x, SIGN_MASK)
      #random_exp_bits = tf.random.uniform(shape=tf.shape(man_bits),maxval=max_exp - zero_threshold, 
      #                                    minval=max_exp -zero_threshold-8, dtype=tf.int32)
      #print("random_exp_bits: ", random_exp_bits)
      # random_exp_bits = bitwise_ops.left_shift(random_exp_bits,23)
      # random_sign_exp_bits = bitwise_ops.bitwise_or(sign_bit, random_exp_bits)

      sign_exp_lsb = bitwise_ops.bitwise_and(convert_x, SIGN_MASK)
      exp_lsb = max_exp - (num_mantissa - 1) - (interval_bool[0] - 1)
      exp_lsb = bitwise_ops.left_shift(exp_lsb, 23)
      sign_exp_lsb = bitwise_ops.bitwise_or(sign_exp_lsb, exp_lsb)

      zeros_tensor = tf.where(exp_only, sign_exp_bits , zeros_tensor  )
      zeros_tensor = tf.where(non_zero_mantissa, sign_exp_lsb, zeros_tensor)

      final_value = tf.where(shift_to_zero, zeros_tensor, final_value)


      #final_value= tf.where(exp_only_new, random_sign_exp_bits, final_value)

      #final_value = bitwise_ops.bitwise_and(final_value, mask)

      x = tf.bitcast(final_value, tf.float32)
      #print("final x: ", x)
      # x = tf.where(guard_bit, x + final_round_float, x )

      if skip_processing == False:
        if is_tensor:
          t_value[t_num] = tf.constant(x)
        else:
          t_value[t_num] = x
      else:
        t_value = x



      #######################################
    elif (round_mode == "afp_sector_update_jamming_1bit"):

      # Use the same exponent per tensor
      # Truncate bits that are shifted off - TODO: Look into rounding

      convert_x = tf.bitcast(x, tf.int32)

      sign_bit = bitwise_ops.bitwise_and(convert_x, SIGN_MASK)
      exp_bits = bitwise_ops.bitwise_and(convert_x, EXP_MASK)
      man_bits = bitwise_ops.bitwise_and(convert_x, MAN_MASK)
      save_man_bits = man_bits

      exp_bits = bitwise_ops.right_shift(exp_bits,23)
      saved_exp = exp_bits

      max_exp = tf.math.reduce_max(exp_bits)
      shifted_max_exp = bitwise_ops.left_shift(max_exp, 23)

      max_tensor = tf.ones_like(exp_bits)
      max_tensor = tf.math.multiply(max_exp, max_tensor)
      diff_tensor = tf.math.subtract(max_tensor,exp_bits)

      ####################
      # Sector Update Code
      # New code to profile intervals
      # print("shifted_man_bits: ", save_man_bits)
      interval_num = []

      for i in range(8):
        interval_num.append(0)

      shifted_man_bits = bitwise_ops.right_shift(save_man_bits, 1)
      shifted_man_bits = bitwise_ops.bitwise_or(shifted_man_bits, NEAREST_PTR)
      shifted_man_bits = bitwise_ops.right_shift(shifted_man_bits, diff_tensor)
      # implicit_one = tf.math.equal(exp_bits, max_exp) 
      # implicit_mask = tf.where(implicit_one, NEAREST_PTR, 0)
      # shifted_man_bits = bitwise_ops.bitwise_or(shifted_man_bits, implicit_mask)
      truncate_mask2 = NEAREST_PTR 
      # for i in range(num_mantissa - 1):
      for i in range(2):
        truncate_mask2 = truncate_mask2 >> 1 | NEAREST_PTR

      #print("truncate_mask2: ", truncate_mask2)
      #print("shifted_man_bits: ", shifted_man_bits)
      interval_bits = bitwise_ops.bitwise_and(shifted_man_bits, truncate_mask2)
      #print("masked shift_man interval_bits:")
      #print(interval_bits)
      interval_bits = bitwise_ops.right_shift(interval_bits, 20)
      interval_int = tf.bitcast(interval_bits, tf.int32)
      #print("interval_bits:")
      #print(interval_bits)
      #print("interval_int: ", interval_bits)

      one_hot_interval = tf.one_hot(interval_int, 8)
      #print("one_hot_interval:")
      #print(one_hot_interval)

      for i in range(8):
        for j in range(len(one_hot_interval)):
          interval_num[i] += one_hot_interval[j].numpy()

      #####
      interval_bool = deepcopy(interval_num[0])
      #print("interval_count: ", interval_bool)
      #interval_bool = deepcopy(interval_num)
      free_encodings = 0
      for i in range(8):
        if interval_bool[i] > 0:
          interval_bool[i] = 1
        else:
          free_encodings += 1
      #print("Free Encodings: ", free_encodings)

      ##############
      # All same sign update
      min_value = tf.math.reduce_min(x)
      max_value = tf.math.reduce_max(x)
      all_positive = min_value >= 0
      all_negative = max_value <= 0
      all_one_sign = all_positive or all_negative

      ##################
      # range code
      abs_x = tf.math.abs(x)
      max_abs_value = tf.math.reduce_max(abs_x)
      min_abs_value = tf.math.reduce_min(abs_x)

      # Code to check if min value in block is > 0001 assuming 3-bit mantissa
      min_over_half = max_exp - 3
      min_over_half = bitwise_ops.left_shift(min_over_half, 23)
      min_over_half = tf.bitcast(min_over_half, tf.float32)
      min_just_over_half = min_abs_value > min_over_half
      # Code to check for round down to 2^max_exp
      
      range_mask1 = int(NEAREST_PTR) >> 2
      # use num_mantissa - 2 to find LSB, use num_mantissa - 1 to find bit to right of LSB to check for rounding down
      #for j in range(0, num_mantissa-1):
      #  range_mask1 = range_mask1 >> 1
      
      # truncate max_exp - instead of checking for 1001 (for 3 bit mantissa)
      # check for 100
      # over_half = shifted_max_exp # bitwise_ops.left_shift(max_exp, 23)

      over_half = bitwise_ops.bitwise_or(shifted_max_exp, range_mask1)

      just_over_half = tf.bitcast(over_half, tf.float32)

      high_exp = max_exp
      if (high_exp != 0):
        high_exp -= 127

      round_down_to_half = max_abs_value < just_over_half
      round_down_to_one_inc = False
      round_down_to_two_inc = False

      extra_num_exp = 0
      max_value_float = 0.0
      max_mantissa = 0

      # If there are gaps in the double precision
      # high threshold describes the upper bound for double precision
      high_threshold = 0.0
      low_threshold = 0.0
      min_mantissa = 0
      high_precision_gap = False

      if (all_one_sign == True):
        # num_mantissa += 1
        free_encodings += 8

      original_num = num_mantissa
      #print("New block - remaining free encodings: ", free_encodings)
      # Code to check for 2x precision - equivalent to 1 extra bit
      if (free_encodings > 0):
        if all_one_sign == False:
          required_encodings = 8 - free_encodings
        else:
          required_encodings = 16 - free_encodings

        while (free_encodings >= required_encodings):
          num_mantissa += 1
          free_encodings -= required_encodings
          required_encodings = 2*required_encodings

      print("num_mantissa: (original), (new) ", original_num, num_mantissa)
      #print("remaining free encodings: ", free_encodings)
      # check for remaining free encodings

      if (free_encodings > 0):
        int_num = 0
        # Need to add code here to selectively increase accuracy for sectors
        # interval_bool = 2 for sections with higher accuracy
        while free_encodings > 0:
          if interval_bool[int_num] >= 1:
            interval_bool[int_num] += 1
            free_encodings -= 1
          int_num = (int_num + 1) % 8

      ############################
      # Create tensor with flags specific to each value to indicate increase in accuracy
      # Flag to indicate select range for higher precision
      # high_accuracy_sector stores the number of extra bits of precision for the sector
      high_accuracy_sector = tf.zeros_like(convert_x)
      for i in range(8):
        temp_bool = tf.math.equal(interval_int, i)
        high_accuracy_sector = tf.where(temp_bool, interval_bool[i] -1 , high_accuracy_sector)

      negative_sector = tf.math.less(high_accuracy_sector, 0)
      high_accuracy_sector = tf.where(negative_sector, 0, high_accuracy_sector)
      print("high_accuracy_sector: ", high_accuracy_sector)
      ############################

      #print("after while loop")
      if (interval_bool[0] > 0):
        zero_threshold = num_mantissa - 1 + interval_bool[0] - 1
      else:
        zero_threshold = num_mantissa - 1  
      shift_to_zero = tf.math.greater(diff_tensor, zero_threshold) 

      # Based on the difference of the exponent, round the bits shifted - start with truncation
      exp_bits = bitwise_ops.left_shift(exp_bits,23)
      man_bits = bitwise_ops.right_shift(man_bits, diff_tensor)

      # Need to mask out the bits lower than the num_mantissa bits
      mask = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa-1):
        mask = mask >> 1
        mask = mask | 0x80000000

      mask1 = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa):
        mask1 = mask1 >> 1
        mask1 = mask1 | 0x80000000

      mask2 = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa + 1):
        mask2 = mask2 >> 1
        mask2 = mask2 | 0x80000000
      
      mask3 = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa + 2):
        mask3 = mask3 >> 1
        mask3 = mask3 | 0x80000000      
      ###########################
      # select high precision code
      tensor_mask = tf.ones_like(convert_x)
      tensor_mask = tf.math.multiply(tensor_mask, mask)
      equal_1 = tf.equal(high_accuracy_sector, 1)
      tensor_mask = tf.where(equal_1, mask1, tensor_mask)
      equal_2 = tf.equal(high_accuracy_sector, 2)
      tensor_mask = tf.where(equal_2, mask2, tensor_mask)
      equal_3 = tf.equal(high_accuracy_sector, 3)
      tensor_mask = tf.where(equal_3, mask3, tensor_mask)   
      #print("mask, 1, 2, 3: ", mask, mask1, mask2, mask3)
      #print("equal_1, 2, 3: ", equal_1, equal_2, equal_3)
      #print("tensor_mask: ", tensor_mask)

      ######### Round Code ###########
      ## Adding jamming code
      ## setting mask with 3 on bits
      guard_mask_save = NEAREST_PTR
      #for i in range(2):
      #  guard_mask_save = guard_mask_save >> 1 |NEAREST_PTR
        
      for j in range(num_mantissa - 1):
        guard_mask = guard_mask_save >> 1

      guard_mask1 = guard_mask_save
      for j in range(num_mantissa):
        guard_mask1 = guard_mask1 >> 1

      guard_mask2 = guard_mask_save
      for j in range(num_mantissa + 1):
        guard_mask2 = guard_mask2 >> 1

      guard_mask3 = guard_mask_save
      for j in range(num_mantissa + 2):
        guard_mask3 = guard_mask3 >> 1
      ###########################
      # select high precision code
      tensor_guard_mask = tf.ones_like(convert_x)
      tensor_guard_mask = tf.math.multiply(tensor_mask, guard_mask)
      #equal_1 = tf.equal(high_accuracy_sector, 1)
      tensor_guard_mask = tf.where(equal_1, guard_mask1, tensor_guard_mask)
      #equal_2 = tf.equal(high_accuracy_sector, 2)
      tensor_guard_mask = tf.where(equal_2, guard_mask2, tensor_guard_mask)
      #equal_3 = tf.equal(high_accuracy_sector, 3)
      tensor_guard_mask = tf.where(equal_3, guard_mask3, tensor_guard_mask)  

      print("tensor_guard_mask: ", tensor_guard_mask)

      ##########################
      # Create a 1 in the lsb of each value (lsb_mask)
      # Use this as a mask to get the actual lsb,
      # zero out this mask based on the condition (actual lsb is 0 and guard bits are all 0)

      lsb_mask = NEAREST_PTR << 1
      for j in range(num_mantissa - 1):
        lsb_mask = lsb_mask >> 1

      lsb_mask1 = NEAREST_PTR << 1
      for j in range(num_mantissa):
        lsb_mask1 = lsb_mask1 >> 1

      lsb_mask2 = NEAREST_PTR << 1
      for j in range(num_mantissa + 1):
        lsb_mask2 = lsb_mask2 >> 1

      lsb_mask3 = NEAREST_PTR << 1
      for j in range(num_mantissa + 2):
        lsb_mask3 = lsb_mask3 >> 1

      tensor_lsb_mask = tf.ones_like(convert_x)
      tensor_lsb_mask = tf.math.multiply(tensor_lsb_mask, lsb_mask)
      #equal_1 = tf.equal(high_accuracy_sector, 1)
      tensor_lsb_mask = tf.where(equal_1, lsb_mask1, tensor_lsb_mask)
      #equal_2 = tf.equal(high_accuracy_sector, 2)
      tensor_lsb_mask = tf.where(equal_2, lsb_mask2, tensor_lsb_mask)
      #equal_3 = tf.equal(high_accuracy_sector, 3)
      tensor_lsb_mask = tf.where(equal_3, lsb_mask3, tensor_lsb_mask)  

      print("tensor_lsb_mask: ", tensor_lsb_mask)
      tensor_lsb_bit = bitwise_ops.bitwise_and(man_bits, tensor_lsb_mask)
      lsb_bit = tf.cast(tensor_lsb_bit, tf.bool)
      print("lsb_bit: ", lsb_bit)
      
      #print("guard_mask ", guard_mask)
      #print("tensor_guard_mask: ", tensor_guard_mask)
      ######### Round Code ###########

      #print("guard_mask: ", guard_mask)
      # guard_bit = bitwise_ops.bitwise_and(man_bits, guard_mask)
      print("man_bits: ", man_bits)
      guard_bit = bitwise_ops.bitwise_and(man_bits, tensor_guard_mask)
      print("3 guard_bits: ", guard_bit)
      ######### Jamming Code
      # guard_bit is true if any of guard bits is a 1 -  jamming
      guard_bit = tf.cast(guard_bit, tf.bool)
      jamming_lsb = tf.logical_or(lsb_bit, guard_bit)
      jamming_mantissa = bitwise_ops.bitwise_or(man_bits, tensor_lsb_mask)
      print("jamming_lsb, mantissa: ", jamming_lsb, jamming_mantissa)
      man_bits = tf.where(jamming_lsb, jamming_mantissa, man_bits)

      not_shift_zero = tf.math.less_equal(diff_tensor, zero_threshold)
      guard_bit = tf.logical_and(guard_bit, not_shift_zero)

      #print("guard_bit: ", guard_bit)

      #### Need to work on this code to enable rounding!!!
      # final_round_exp = tf.math.subtract(saved_exp, num_mantissa - 1)
      '''
      final_round_exp = tf.math.subtract(saved_exp, num_mantissa - 1)
      final_round_exp = tf.math.subtract(final_round_exp, high_accuracy_sector)
      final_round_exp = tf.math.add(final_round_exp, diff_tensor) 
      final_round_exp = bitwise_ops.left_shift(final_round_exp,23)
      final_round_value = bitwise_ops.bitwise_or(sign_bit, final_round_exp)
      final_round_float = tf.bitcast(final_round_value, tf.float32)
      '''
      #print("final_round_float: ", final_round_float)
      
      ###############################

      # man_bits = bitwise_ops.bitwise_and(man_bits, mask)
      man_bits = bitwise_ops.bitwise_and(man_bits, tensor_mask)
      man_bits = bitwise_ops.left_shift(man_bits, diff_tensor)

      final_value = bitwise_ops.bitwise_or(sign_bit,exp_bits)
      final_value = bitwise_ops.bitwise_or(final_value, man_bits)

      zeros_tensor = tf.zeros_like(man_bits)
      
      exp_only = tf.math.greater(diff_tensor, num_mantissa - 1)
      not_shift_zero = tf.math.less_equal(diff_tensor, zero_threshold)
      exp_only = tf.logical_and(exp_only, not_shift_zero)
      sign_exp_bits = bitwise_ops.bitwise_and(convert_x, SIGN_EXP_MASK)
      zeros_tensor = tf.where(exp_only, sign_exp_bits , zeros_tensor  )

      final_value = tf.where(shift_to_zero, zeros_tensor, final_value)

      #final_value = bitwise_ops.bitwise_and(final_value, mask)

      x = tf.bitcast(final_value, tf.float32)

      # x = tf.where(guard_bit, x + final_round_float, x )

      if skip_processing == False:
        if is_tensor:
          t_value[t_num] = tf.constant(x)
        else:
          t_value[t_num] = x
      else:
        t_value = x


      #######################################
            
      
    elif (round_mode == "afp_sector_update"):

      # Use the same exponent per tensor
      # Truncate bits that are shifted off - TODO: Look into rounding

      convert_x = tf.bitcast(x, tf.int32)

      sign_bit = bitwise_ops.bitwise_and(convert_x, SIGN_MASK)
      exp_bits = bitwise_ops.bitwise_and(convert_x, EXP_MASK)
      man_bits = bitwise_ops.bitwise_and(convert_x, MAN_MASK)
      save_man_bits = man_bits

      exp_bits = bitwise_ops.right_shift(exp_bits,23)
      saved_exp = exp_bits

      max_exp = tf.math.reduce_max(exp_bits)
      shifted_max_exp = bitwise_ops.left_shift(max_exp, 23)

      max_tensor = tf.ones_like(exp_bits)
      max_tensor = tf.math.multiply(max_exp, max_tensor)
      diff_tensor = tf.math.subtract(max_tensor,exp_bits)

      ####################
      # Sector Update Code
      # New code to profile intervals
      # print("shifted_man_bits: ", save_man_bits)
      interval_num = []

      for i in range(8):
        interval_num.append(0)

      shifted_man_bits = bitwise_ops.right_shift(save_man_bits, 1)
      shifted_man_bits = bitwise_ops.bitwise_or(shifted_man_bits, NEAREST_PTR)
      shifted_man_bits = bitwise_ops.right_shift(shifted_man_bits, diff_tensor)
      # implicit_one = tf.math.equal(exp_bits, max_exp) 
      # implicit_mask = tf.where(implicit_one, NEAREST_PTR, 0)
      # shifted_man_bits = bitwise_ops.bitwise_or(shifted_man_bits, implicit_mask)
      truncate_mask2 = NEAREST_PTR 
      # for i in range(num_mantissa - 1):
      for i in range(2):
        truncate_mask2 = truncate_mask2 >> 1 | NEAREST_PTR

      #print("truncate_mask2: ", truncate_mask2)
      #print("shifted_man_bits: ", shifted_man_bits)
      interval_bits = bitwise_ops.bitwise_and(shifted_man_bits, truncate_mask2)
      #print("masked shift_man interval_bits:")
      #print(interval_bits)
      interval_bits = bitwise_ops.right_shift(interval_bits, 20)
      interval_int = tf.bitcast(interval_bits, tf.int32)
      #print("interval_bits:")
      #print(interval_bits)
      #print("interval_int: ", interval_bits)

      one_hot_interval = tf.one_hot(interval_int, 8)
      #print("one_hot_interval:")
      #print(one_hot_interval)

      for i in range(8):
        for j in range(len(one_hot_interval)):
          interval_num[i] += one_hot_interval[j].numpy()

      #####
      interval_bool = deepcopy(interval_num[0])
      #interval_bool = deepcopy(interval_num)
      free_encodings = 0
      for i in range(8):
        if interval_bool[i] > 0:
          interval_bool[i] = 1
        else:
          free_encodings += 1
      #print("Free Encodings: ", free_encodings)

      ##############
      # All same sign update
      min_value = tf.math.reduce_min(x)
      max_value = tf.math.reduce_max(x)
      all_positive = min_value >= 0
      all_negative = max_value <= 0
      all_one_sign = all_positive or all_negative

      ##################
      # range code
      abs_x = tf.math.abs(x)
      max_abs_value = tf.math.reduce_max(abs_x)
      min_abs_value = tf.math.reduce_min(abs_x)

      # Code to check if min value in block is > 0001 assuming 3-bit mantissa
      min_over_half = max_exp - 3
      min_over_half = bitwise_ops.left_shift(min_over_half, 23)
      min_over_half = tf.bitcast(min_over_half, tf.float32)
      min_just_over_half = min_abs_value > min_over_half
      # Code to check for round down to 2^max_exp
      
      range_mask1 = int(NEAREST_PTR) >> 2
      # use num_mantissa - 2 to find LSB, use num_mantissa - 1 to find bit to right of LSB to check for rounding down
      #for j in range(0, num_mantissa-1):
      #  range_mask1 = range_mask1 >> 1
      
      # truncate max_exp - instead of checking for 1001 (for 3 bit mantissa)
      # check for 100
      # over_half = shifted_max_exp # bitwise_ops.left_shift(max_exp, 23)

      over_half = bitwise_ops.bitwise_or(shifted_max_exp, range_mask1)

      just_over_half = tf.bitcast(over_half, tf.float32)

      high_exp = max_exp
      if (high_exp != 0):
        high_exp -= 127

      round_down_to_half = max_abs_value < just_over_half
      round_down_to_one_inc = False
      round_down_to_two_inc = False

      extra_num_exp = 0
      max_value_float = 0.0
      max_mantissa = 0

      # If there are gaps in the double precision
      # high threshold describes the upper bound for double precision
      high_threshold = 0.0
      low_threshold = 0.0
      min_mantissa = 0
      high_precision_gap = False

      if (all_one_sign == True):
        # num_mantissa += 1
        free_encodings += 8

      original_num = num_mantissa
      #print("New block - remaining free encodings: ", free_encodings)
      # Code to check for 2x precision - equivalent to 1 extra bit
      if (free_encodings > 0):
        if all_one_sign == False:
          required_encodings = 8 - free_encodings
        else:
          required_encodings = 16 - free_encodings

        while (free_encodings >= required_encodings):
          num_mantissa += 1
          free_encodings -= required_encodings
          required_encodings = 2*required_encodings

      #print("num_mantissa: (original), (new) ", original_num, num_mantissa)
      #print("remaining free encodings: ", free_encodings)
      # check for remaining free encodings

      if (free_encodings > 0):
        int_num = 0
        # Need to add code here to selectively increase accuracy for sectors
        # interval_bool = 2 for sections with higher accuracy
        while free_encodings > 0:
          if interval_bool[int_num] >= 1:
            interval_bool[int_num] += 1
            free_encodings -= 1
          int_num = (int_num + 1) % 8

      ############################
      # Create tensor with flags specific to each value to indicate increase in accuracy
      # Flag to indicate select range for higher precision
      # high_accuracy_sector stores the number of extra bits of precision for the sector
      high_accuracy_sector = tf.zeros_like(convert_x)
      for i in range(8):
        temp_bool = tf.math.equal(interval_int, i)
        high_accuracy_sector = tf.where(temp_bool, interval_bool[i] -1 , high_accuracy_sector)

      negative_sector = tf.math.less(high_accuracy_sector, 0)
      high_accuracy_sector = tf.where(negative_sector, 0, high_accuracy_sector)
      #print("high_accuracy_sector: ", high_accuracy_sector)
      ############################

      #print("after while loop")
      if (interval_bool[0] > 0):
        zero_threshold = num_mantissa - 1 + interval_bool[0] - 1
      else:
        zero_threshold = num_mantissa - 1  
      shift_to_zero = tf.math.greater(diff_tensor, zero_threshold) 

      # Based on the difference of the exponent, round the bits shifted - start with truncation
      exp_bits = bitwise_ops.left_shift(exp_bits,23)
      man_bits = bitwise_ops.right_shift(man_bits, diff_tensor)

      # Need to mask out the bits lower than the num_mantissa bits
      mask = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa-1):
        mask = mask >> 1
        mask = mask | 0x80000000

      mask1 = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa):
        mask1 = mask1 >> 1
        mask1 = mask1 | 0x80000000

      mask2 = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa + 1):
        mask2 = mask2 >> 1
        mask2 = mask2 | 0x80000000
      
      mask3 = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa + 2):
        mask3 = mask3 >> 1
        mask3 = mask3 | 0x80000000      
      ###########################
      # select high precision code
      tensor_mask = tf.ones_like(convert_x)
      tensor_mask = tf.math.multiply(tensor_mask, mask)
      equal_1 = tf.equal(high_accuracy_sector, 1)
      tensor_mask = tf.where(equal_1, mask1, tensor_mask)
      equal_2 = tf.equal(high_accuracy_sector, 2)
      tensor_mask = tf.where(equal_2, mask2, tensor_mask)
      equal_3 = tf.equal(high_accuracy_sector, 3)
      tensor_mask = tf.where(equal_3, mask3, tensor_mask)   
      #print("mask, 1, 2, 3: ", mask, mask1, mask2, mask3)
      #print("equal_1, 2, 3: ", equal_1, equal_2, equal_3)
      #print("tensor_mask: ", tensor_mask)

      ######### Round Code ###########
      guard_mask = NEAREST_PTR
      for j in range(num_mantissa - 1):
        guard_mask = guard_mask >> 1

      guard_mask1 = NEAREST_PTR
      for j in range(num_mantissa):
        guard_mask1 = guard_mask1 >> 1

      guard_mask2 = NEAREST_PTR
      for j in range(num_mantissa + 1):
        guard_mask2 = guard_mask2 >> 1

      guard_mask3 = NEAREST_PTR
      for j in range(num_mantissa + 2):
        guard_mask3 = guard_mask3 >> 1
      ###########################
      # select high precision code
      tensor_guard_mask = tf.ones_like(convert_x)
      tensor_guard_mask = tf.math.multiply(tensor_mask, guard_mask)
      equal_1 = tf.equal(high_accuracy_sector, 1)
      tensor_guard_mask = tf.where(equal_1, guard_mask1, tensor_guard_mask)
      equal_2 = tf.equal(high_accuracy_sector, 2)
      tensor_guard_mask = tf.where(equal_2, guard_mask2, tensor_guard_mask)
      equal_3 = tf.equal(high_accuracy_sector, 3)
      tensor_guard_mask = tf.where(equal_3, guard_mask3, tensor_guard_mask)  

      #print("guard_mask ", guard_mask)
      #print("tensor_guard_mask: ", tensor_guard_mask)
      ######### Round Code ###########

      #print("guard_mask: ", guard_mask)
      # guard_bit = bitwise_ops.bitwise_and(man_bits, guard_mask)
      guard_bit = bitwise_ops.bitwise_and(man_bits, tensor_guard_mask)
      guard_bit = tf.cast(guard_bit, tf.bool)
      not_shift_zero = tf.math.less_equal(diff_tensor, zero_threshold)
      guard_bit = tf.logical_and(guard_bit, not_shift_zero)

      #print("guard_bit: ", guard_bit)

      #### Need to work on this code to enable rounding!!!
      # final_round_exp = tf.math.subtract(saved_exp, num_mantissa - 1)

      final_round_exp = tf.math.subtract(saved_exp, num_mantissa - 1)
      final_round_exp = tf.math.subtract(final_round_exp, high_accuracy_sector)
      final_round_exp = tf.math.add(final_round_exp, diff_tensor) 
      final_round_exp = bitwise_ops.left_shift(final_round_exp,23)
      final_round_value = bitwise_ops.bitwise_or(sign_bit, final_round_exp)
      final_round_float = tf.bitcast(final_round_value, tf.float32)
      
      #print("final_round_float: ", final_round_float)
      
      ###############################

      # man_bits = bitwise_ops.bitwise_and(man_bits, mask)
      man_bits = bitwise_ops.bitwise_and(man_bits, tensor_mask)
      man_bits = bitwise_ops.left_shift(man_bits, diff_tensor)

      final_value = bitwise_ops.bitwise_or(sign_bit,exp_bits)
      final_value = bitwise_ops.bitwise_or(final_value, man_bits)

      zeros_tensor = tf.zeros_like(man_bits)
      
      exp_only = tf.math.greater(diff_tensor, num_mantissa - 1)
      not_shift_zero = tf.math.less_equal(diff_tensor, zero_threshold)
      exp_only = tf.logical_and(exp_only, not_shift_zero)
      sign_exp_bits = bitwise_ops.bitwise_and(convert_x, SIGN_EXP_MASK)
      zeros_tensor = tf.where(exp_only, sign_exp_bits , zeros_tensor  )

      final_value = tf.where(shift_to_zero, zeros_tensor, final_value)

      #final_value = bitwise_ops.bitwise_and(final_value, mask)

      x = tf.bitcast(final_value, tf.float32)

      x = tf.where(guard_bit, x + final_round_float, x )

      if skip_processing == False:
        if is_tensor:
          t_value[t_num] = tf.constant(x)
        else:
          t_value[t_num] = x
      else:
        t_value = x


    elif (round_mode == "afp_optimal_range_msfp_nearest"):

      # Use the same exponent per tensor
      # Truncate bits that are shifted off - TODO: Look into rounding

      convert_x = tf.bitcast(x, tf.int32)
      #print("x: ", x)

      sign_bit = bitwise_ops.bitwise_and(convert_x, SIGN_MASK)
      exp_bits = bitwise_ops.bitwise_and(convert_x, EXP_MASK)
      man_bits = bitwise_ops.bitwise_and(convert_x, MAN_MASK)
      save_man_bits = man_bits

      exp_bits = bitwise_ops.right_shift(exp_bits,23)
      saved_exp = exp_bits

      max_exp = tf.math.reduce_max(exp_bits)
      shifted_max_exp = bitwise_ops.left_shift(max_exp, 23)

      max_tensor = tf.ones_like(exp_bits)
      max_tensor = tf.math.multiply(max_exp, max_tensor)
      diff_tensor = tf.math.subtract(max_tensor,exp_bits)

      temp_num_mantissa = num_mantissa + 1
      inside_inc1_range = tf.zeros_like(exp_bits)
      inside_inc1_range = tf.cast(inside_inc1_range, tf.bool)
      inside_inc2_range = tf.zeros_like(exp_bits)
      inside_inc2_range = tf.cast(inside_inc2_range, tf.bool)

      ##############
      # All same sign update
      min_value = tf.math.reduce_min(x)
      max_value = tf.math.reduce_max(x)
      all_positive = min_value >= 0
      all_negative = max_value <= 0
      all_one_sign = all_positive or all_negative

      ##################
      # range code
      abs_x = tf.math.abs(x)
      max_abs_value = tf.math.reduce_max(abs_x)
      min_abs_value = tf.math.reduce_min(abs_x)

      # Code to check if min value in block is > 0001 assuming 3-bit mantissa
      min_over_half = max_exp - 3
      min_over_half = bitwise_ops.left_shift(min_over_half, 23)
      min_over_half = tf.bitcast(min_over_half, tf.float32)
      min_just_over_half = min_abs_value > min_over_half
      # Code to check for round down to 2^max_exp
      
      range_mask1 = int(NEAREST_PTR) >> 2
      # use num_mantissa - 2 to find LSB, use num_mantissa - 1 to find bit to right of LSB to check for rounding down
      #for j in range(0, num_mantissa-1):
      #  range_mask1 = range_mask1 >> 1
      
      # truncate max_exp - instead of checking for 1001 (for 3 bit mantissa)
      # check for 100
      # over_half = shifted_max_exp # bitwise_ops.left_shift(max_exp, 23)

      over_half = bitwise_ops.bitwise_or(shifted_max_exp, range_mask1)

      just_over_half = tf.bitcast(over_half, tf.float32)

      high_exp = max_exp
      if (high_exp != 0):
        high_exp -= 127

      round_down_to_half = max_abs_value < just_over_half
      round_down_to_one_inc = False
      round_down_to_two_inc = False

      extra_num_exp = 0
      max_value_float = 0.0
      max_mantissa = 0

      # If there are gaps in the double precision
      # high threshold describes the upper bound for double precision
      high_threshold = 0.0
      low_threshold = 0.0
      min_mantissa = 0
      high_precision_gap = False

      if (all_one_sign == True):
          num_mantissa += 1

      if round_down_to_half == True:
        # approximation on double precision
        # Instead of doubling precision for all values - try capturing smaller values

        # New code for max value float
        max_value = shifted_max_exp
        max_value_float = tf.bitcast(max_value, tf.float32)

        max_mantissa = 0
        if (min_just_over_half == True): 
          num_mantissa += 1

        else:
          num_mantissa += 1

          range_mask1 = NEAREST_PTR
          if (num_mantissa -1 >=3):
            for i in range(num_mantissa -1 -3):
              range_mask1 = range_mask1 >> 1 | NEAREST_PTR

          low_thresh_exp = max_exp -1
          low_thresh_exp = bitwise_ops.left_shift(low_thresh_exp, 23)
          low_threshold = bitwise_ops.bitwise_or(low_thresh_exp, range_mask1)
          low_precision_threshold = tf.bitcast(low_threshold, tf.float32)

          high_precision_gap = True
          #print("high_precision_gap == True")

        # keeping values with exponent only for 1/16, 1/32, 1/64
        #extra_num_exp = 3

      else:
        # check for round down to max_exp + 1 increment

        range_mask1 = int(NEAREST_PTR) >> 1| NEAREST_PTR
        range_mask1 = range_mask1 >> 1 
        
        #range_mask1 = range_mask1 | NEAREST_PTR
        #for j in range(0, num_mantissa-3):        
        #  range_mask1 = range_mask1 >> 1
        # use mantissa of 011 to check

        #round_down = shifted_max_exp # bitwise_ops.left_shift(max_exp, 23)
        round_down = bitwise_ops.bitwise_or(shifted_max_exp, range_mask1)
        round_down_float = tf.bitcast(round_down, tf.float32)        

        round_down_to_one_inc = max_abs_value < round_down_float

        if round_down_to_one_inc == True:
          #extra_num_exp += 2* (2 ** (num_mantissa -3))
          if (min_just_over_half == True):
            '''
            if (num_mantissa < 5):
              extra_num_exp += 1 
            else:
              extra_num_exp += 2 ** (num_mantissa -4)
            '''
            num_mantissa += 1

            range_mask1 = NEAREST_PTR
            if (num_mantissa -1 >=3):
              for i in range(num_mantissa -1 -3):
                range_mask1 = range_mask1 >> 1 | NEAREST_PTR

            low_thresh_exp = max_exp -1
            low_thresh_exp = bitwise_ops.left_shift(low_thresh_exp, 23)
            low_threshold = bitwise_ops.bitwise_or(low_thresh_exp, range_mask1)
            low_precision_threshold = tf.bitcast(low_threshold, tf.float32)
            high_precision_gap = True   

          else:
            num_mantissa += 1

            low_thresh_exp = max_exp -1
            low_thresh_exp = bitwise_ops.left_shift(low_thresh_exp, 23)
            low_threshold = bitwise_ops.bitwise_or(low_thresh_exp, range_mask1)
            low_precision_threshold = tf.bitcast(low_threshold, tf.float32)
            high_precision_gap = True  

          temp_man = NEAREST_PTR >> 1
          max_value = shifted_max_exp | temp_man
          max_value_float = tf.bitcast(max_value, tf.float32)

          # max_value_float = round_down_float
          # max_mantissa = int(NEAREST_PTR)
          # max_mantissa = max_mantissa >> 1
       
        else:
          range_mask1 = int(NEAREST_PTR) | NEAREST_PTR >> 2
          #range_mask1 = range_mask1 >> 1
          #range_mask1 = range_mask1 | NEAREST_PTR
          
          #for j in range(0, num_mantissa-3):        
          #  range_mask1 = range_mask1 >> 1
                        # use mantissa of 101 to check

          #round_down = shifted_max_exp #bitwise_ops.left_shift(max_exp, 23)
          round_down = bitwise_ops.bitwise_or(shifted_max_exp, range_mask1)
          round_down_float = tf.bitcast(round_down, tf.float32)        

          round_down_to_two_inc = max_abs_value < round_down_float

          if round_down_to_two_inc == True:
            #extra_num_exp += 1* (2 ** (num_mantissa -3))
            if (min_just_over_half == True):
              num_mantissa += 1

              low_thresh_exp = max_exp -1
              low_thresh_exp = bitwise_ops.left_shift(low_thresh_exp, 23)
              low_threshold = bitwise_ops.bitwise_or(low_thresh_exp, range_mask1)
              low_precision_threshold = tf.bitcast(low_threshold, tf.float32)
              high_precision_gap = True  
            else:
              extra_num_exp += 1* (2 ** (num_mantissa -3))
          else:
            if (min_just_over_half == True):
              if (num_mantissa < 5):
                extra_num_exp += 1 
              else:
                extra_num_exp += 2 ** (num_mantissa -4)            
            

            temp_man = NEAREST_PTR
            max_value = shifted_max_exp | temp_man
            max_value_float = tf.bitcast(max_value, tf.float32)

      round_down_flag = tf.math.logical_or(round_down_to_half, round_down_to_one_inc)
      round_down_flag = tf.math.logical_or(round_down_flag, round_down_to_two_inc)
      #########################
      # Add code to round numbers based on reduced range before passing
      #########################

      # Allow a 1 in the LSB-1 position to be rounded up
      # TODO: Need to check its's 11
      new_shift_to_lsb = tf.math.equal(diff_tensor, num_mantissa + extra_num_exp)
      new_shift_to_lsb_guard = bitwise_ops.bitwise_and(NEAREST_PTR, save_man_bits)
      new_shift_to_lsb_guard = tf.cast(new_shift_to_lsb_guard, tf.bool)
      new_shift_to_lsb = tf.math.logical_and(new_shift_to_lsb, new_shift_to_lsb_guard)

      new_shift_to_lsb_value = saved_exp + 1
      new_shift_to_lsb_value = bitwise_ops.left_shift(new_shift_to_lsb_value, 23)
      new_shift_to_lsb_value = bitwise_ops.bitwise_or(new_shift_to_lsb_value, sign_bit)


      low_exp_only_range = tf.math.greater(diff_tensor, num_mantissa - 1)
      high_exp_only_range = tf.math.less(diff_tensor, num_mantissa + extra_num_exp)
      in_exp_only_range = tf.math.logical_and(low_exp_only_range, high_exp_only_range)

      new_shift_to_zero = tf.math.greater(diff_tensor, num_mantissa-1 + extra_num_exp) 
      shift_to_zero = tf.math.greater(diff_tensor, num_mantissa-1)
      #print("diff_tensor: ", diff_tensor)
      #print("num_mantissa: ", num_mantissa)

      #save_exp_bits = exp_bits
      # Based on the difference of the exponent, round the bits shifted - start with truncation
      temp_exp_bits = bitwise_ops.left_shift(exp_bits,23)
      temp_man_bits = bitwise_ops.right_shift(man_bits, diff_tensor)

      #saved_man_bits = man_bits
      # Need to mask out the bits lower than the num_mantissa bits
      mask = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa-1):
        mask = mask >> 1
        mask = mask | 0x80000000
      
      temp_man_bits = bitwise_ops.bitwise_and(temp_man_bits, mask)

      temp_man_bits = bitwise_ops.left_shift(temp_man_bits, diff_tensor)

      final_value = bitwise_ops.bitwise_or(sign_bit,temp_exp_bits)
      final_value = bitwise_ops.bitwise_or(final_value, temp_man_bits)

      final_float = tf.bitcast(final_value, tf.float32)

      ###

      exp_only_value = bitwise_ops.bitwise_and(convert_x, SIGN_EXP_MASK)
      exp_only_float = tf.bitcast(exp_only_value, tf.float32)

      exp_only_guard = bitwise_ops.bitwise_and(NEAREST_PTR, save_man_bits)
      exp_only_guard = tf.cast(exp_only_guard, tf.bool)

      exp_only_round_float = tf.math.add(exp_only_float, exp_only_float)
      exp_only_float = tf.where(exp_only_guard, exp_only_round_float, exp_only_float)

      #print("Final Float after truncation:" , final_float)
      ##################################
      # Increased precision range code
      '''
      mask = int(TRUNCATE_MASK)
      for j in range(0, temp_num_mantissa-1):
        mask = mask >> 1
        mask = mask | 0x80000000

      temp_man_bits = bitwise_ops.bitwise_and(saved_man_bits, mask)
      temp_man_bits = bitwise_ops.left_shift(temp_man_bits, diff_tensor)

      temp_value = bitwise_ops.bitwise_or(sign_bit,exp_bits)
      temp_value = bitwise_ops.bitwise_or(temp_value, temp_man_bits)

      temp_float = tf.bitcast(temp_value, tf.float32)
      final_float = tf.where(inside_inc1_range, temp_float, final_float)
      final_float = tf.where(inside_inc2_range, temp_float, final_float)

      # Need to check for round up      
      '''
      ###################################
      # Add nearest rounding code

      guard_bit1 = int(NEAREST_PTR)
     
      for j in range(0, num_mantissa-1):        
        guard_bit1 = guard_bit1 >> 1
      
      guard_bit2 = guard_bit1 >> 1

      save_final_float = final_float

      #final_guard = tf.ones_like(exp_bits)
      final_guard = tf.where(inside_inc1_range, guard_bit2, guard_bit1)
      final_guard = tf.where(inside_inc2_range, guard_bit2, final_guard)     
      final_guard = bitwise_ops.left_shift(final_guard, diff_tensor)
      saved_final_guard = final_guard

      #print("final_guard: ", final_guard)
      #print("save_man_bits: ", save_man_bits)
      final_guard = bitwise_ops.bitwise_and(final_guard, save_man_bits)

      implicit_one_guard = tf.math.equal(diff_tensor, num_mantissa)

      #  Comment out to allow extra exponent values to work
      #shift_to_zero = tf.where(implicit_one_guard, False, shift_to_zero)

      #implicit_one_guard = bitwise_ops.bitwise_and(saved_final_guard, MAN_OVERFLOW )
      #implicit_one_guard = tf.cast(implicit_one_guard, tf.bool)
      #print("implicit_one_guard: ", implicit_one_guard)
      #print("diff tensor: ", diff_tensor)
      #print("num_mantissa: ", num_mantissa)

      implicit_round_value = bitwise_ops.bitwise_and(convert_x, SIGN_EXP_MASK)
      implicit_round_float = tf.bitcast(implicit_round_value, tf.float32)
      #print("implicit_round_float: ", implicit_round_float)

      #final_guard = bitwise_ops.bitwise_and(final_guard, MAN_MASK)
      final_guard = tf.cast(final_guard, tf.bool)
      #print("final_guard: ", final_guard)

      final_round_exp = tf.math.subtract(saved_exp, num_mantissa - 1)
      final_round_exp = tf.math.add(final_round_exp, diff_tensor) 
      final_round_exp = bitwise_ops.left_shift(final_round_exp,23)
      final_round_value = bitwise_ops.bitwise_or(sign_bit, final_round_exp)
      final_round_float = tf.bitcast(final_round_value, tf.float32)
      #print("final_round_float: ", final_round_float)

      intermediate_float = tf.where(implicit_one_guard, final_float + implicit_round_float, final_float)
      intermediate_float = tf.where(final_guard, final_float + final_round_float, intermediate_float)

      # Cap max value at the same max exponent, instead of carry out into higher max exp
      check_value = tf.bitcast(intermediate_float, tf.int32)
      new_exp_bits = bitwise_ops.bitwise_and(check_value, EXP_MASK)
      new_exp_bits = bitwise_ops.right_shift(new_exp_bits, 23)
      round_down = tf.math.greater(new_exp_bits, max_exp)

      final_float = tf.where(round_down, final_float, intermediate_float)

      final_value = tf.bitcast(final_float, tf.int32)
      #################################
      zeros_tensor = tf.zeros_like(man_bits)
      zeros_tensor = tf.where(new_shift_to_zero, zeros_tensor, exp_only_value)
      zeros_tensor = tf.where(new_shift_to_lsb, new_shift_to_lsb_value, zeros_tensor)
      zeros_tensor = tf.where(in_exp_only_range, exp_only_value, zeros_tensor)

      #print("new_shift_to_zero", new_shift_to_zero)
      #print("new_shift_to_lsb ", new_shift_to_lsb)
      #print("in_exp_only_range: ", in_exp_only_range)
      #print("shift_to_zero: ", shift_to_zero)
      #print("zeros_tensor: ", zeros_tensor)
      final_value = tf.where(shift_to_zero, zeros_tensor, final_value)

      if (round_down_flag == True):
        pass
        '''
        # This needs to take care of sign too!
        temp_final_float = tf.bitcast(final_value, tf.float32)
        temp_abs_float = tf.math.abs(temp_final_float)
        select_round_down = tf.math.greater(temp_abs_float, max_value_float)
        temp_max_value = tf.bitcast(max_value_float, tf.int32)
        temp_max_value = bitwise_ops.bitwise_or(temp_max_value, sign_bit)
        final_value = tf.where(select_round_down, temp_max_value, final_value)        
        '''

      if (high_precision_gap == True):
        temp_final_float = tf.bitcast(final_value, tf.float32)
        #print("num_mantissa: ", num_mantissa)
        #print("x: ", temp_final_float)
        temp_abs_float = tf.math.abs(temp_final_float)
        select_lower_precision = tf.math.greater_equal(temp_abs_float, low_precision_threshold)
        
        temp_man = bitwise_ops.bitwise_and(final_value, MAN_MASK)
        temp_man = bitwise_ops.right_shift(temp_man, diff_tensor)

        mask = int(TRUNCATE_MASK)

        for j in range(0, num_mantissa-2):
          mask = mask >> 1
          mask = mask | 0x80000000

        temp_man = bitwise_ops.bitwise_and(temp_man, mask)
        temp_man = bitwise_ops.left_shift(temp_man, diff_tensor)

        low_precision_value = bitwise_ops.bitwise_and(final_value, SIGN_EXP_MASK)
        low_precision_value = bitwise_ops.bitwise_or(temp_man, low_precision_value)
        #low_precision_value = bitwise_ops.bitwise_and(final_value, mask)

        #print("select_round_down: ", select_round_down)
        # need to incorporate sign bit
        temp_final_value = tf.where(select_lower_precision, low_precision_value, final_value)

        final_value = temp_final_value           

      final_float = tf.bitcast(final_value, tf.float32)
      #print("final x: ", final_float)
      '''
      abs_final_float = tf.math.abs(final_float)

      cap_to_max = tf.math.greater(abs_final_float, max_value_float)
      new_final_value = bitwise_ops.bitwise_and(final_value, SIGN_EXP_MASK)
      og_mantissa = bitwise_ops.bitwise_and(final_value, MAN_MASK)
      new_final_mantissa = tf.where(cap_to_max, max_mantissa, og_mantissa )
      new_final_value = bitwise_ops.bitwise_or(new_final_value, new_final_mantissa)
      '''
      x = tf.bitcast(final_value, tf.float32)

      if skip_processing == False:
        if is_tensor:
          t_value[t_num] = tf.constant(x)
        else:
          t_value[t_num] = x
      else:
        t_value = x

    elif (round_mode == "msfp_exponent_reuse_nearest"):

      # Use the same exponent per tensor
      # Truncate bits that are shifted off - TODO: Look into rounding

      convert_x = tf.bitcast(x, tf.int32)

      sign_bit = bitwise_ops.bitwise_and(convert_x, SIGN_MASK)
      exp_bits = bitwise_ops.bitwise_and(convert_x, EXP_MASK)
      man_bits = bitwise_ops.bitwise_and(convert_x, MAN_MASK)
      save_man_bits = man_bits

      exp_bits = bitwise_ops.right_shift(exp_bits,23)
      saved_exp = exp_bits

      max_exp = tf.math.reduce_max(exp_bits)
      max_tensor = tf.ones_like(exp_bits)
      max_tensor = tf.math.multiply(max_exp, max_tensor)
      diff_tensor = tf.math.subtract(max_tensor,exp_bits)

      shift_to_zero = tf.math.greater(diff_tensor, num_mantissa-1) 

      save_exp_bits = exp_bits
      # Based on the difference of the exponent, round the bits shifted - start with truncation
      exp_bits = bitwise_ops.left_shift(exp_bits,23)
      man_bits = bitwise_ops.right_shift(man_bits, diff_tensor)

      # Need to mask out the bits lower than the num_mantissa bits
      mask = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa-1):
        mask = mask >> 1
        mask = mask | 0x80000000
      
      man_bits = bitwise_ops.bitwise_and(man_bits, mask)

      man_bits = bitwise_ops.left_shift(man_bits, diff_tensor)

      final_value = bitwise_ops.bitwise_or(sign_bit,exp_bits)
      final_value = bitwise_ops.bitwise_or(final_value, man_bits)

      final_float = tf.bitcast(final_value, tf.float32)

      ###################################
      # Add nearest rounding code

      new_round_exp = save_exp_bits - (num_mantissa -1) + diff_tensor
      new_round_exp = bitwise_ops.left_shift(new_round_exp, 23)
      new_round_value = bitwise_ops.bitwise_or(new_round_exp, sign_bit)
      new_round_float = tf.bitcast(new_round_value, tf.float32)
      #print("new_round_float: ", new_round_float)

      second_range_less = tf.math.less(diff_tensor, num_mantissa - 1)
      second_range_round = second_range_less
      #print("second_range_round: ", second_range_round)

      mask4 = tf.ones_like(exp_bits)
      mask4 = mask4 * int(NEAREST_PTR)
      mask4 = bitwise_ops.right_shift(mask4, num_mantissa - diff_tensor -1 )
      #print("mask4: ", mask4)

      zeros_tensor = tf.zeros_like(man_bits)
      half_one2 = bitwise_ops.bitwise_and(mask4, save_man_bits)
      half_one2_true = tf.cast(half_one2, tf.bool)

      # need to look for highest exponent - not max_exp
      #half_one2_true = tf.math.logical_and(not_equal_max, half_one2_true)

      half_one2_true = tf.math.logical_and(half_one2_true,second_range_round)
      #print("half_one2: ", half_one2)
      #print("half_one2_true: ", half_one2_true)

      #not_equal_max = tf.math.not_equal(saved_exp, max_exp)

      intermediate_float = tf.where(half_one2_true, final_float + new_round_float, final_float)
      check_value = tf.bitcast(intermediate_float, tf.int32)
      new_exp_bits = bitwise_ops.bitwise_and(check_value, EXP_MASK)
      new_exp_bits = bitwise_ops.right_shift(new_exp_bits, 23)
      round_down = tf.math.greater(new_exp_bits, max_exp)

      final_float = tf.where(round_down, final_float, intermediate_float)
      final_value = tf.bitcast(final_float, tf.int32)
      #################################

      zeros_tensor = tf.zeros_like(man_bits)
      final_value = tf.where(shift_to_zero, zeros_tensor, final_value)

      final_value = bitwise_ops.bitwise_and(final_value, mask)

      x = tf.bitcast(final_value, tf.float32)

      if skip_processing == False:
        if is_tensor:
          t_value[t_num] = tf.constant(x)
        else:
          t_value[t_num] = x
      else:
        t_value = x


    elif (round_mode == "msfp_exponent_reuse"):

      # Use the same exponent per tensor
      # Truncate bits that are shifted off - TODO: Look into rounding

      convert_x = tf.bitcast(x, tf.int32)

      sign_bit = bitwise_ops.bitwise_and(convert_x, SIGN_MASK)
      exp_bits = bitwise_ops.bitwise_and(convert_x, EXP_MASK)
      man_bits = bitwise_ops.bitwise_and(convert_x, MAN_MASK)

      exp_bits = bitwise_ops.right_shift(exp_bits,23)
      max_exp = tf.math.reduce_max(exp_bits)
      max_tensor = tf.ones_like(exp_bits)
      max_tensor = tf.math.multiply(max_exp, max_tensor)
      diff_tensor = tf.math.subtract(max_tensor,exp_bits)

      shift_to_zero = tf.math.greater(diff_tensor, num_mantissa-1) 

      # Based on the difference of the exponent, round the bits shifted - start with truncation
      exp_bits = bitwise_ops.left_shift(exp_bits,23)
      man_bits = bitwise_ops.right_shift(man_bits, diff_tensor)

      # Need to mask out the bits lower than the num_mantissa bits
      mask = int(TRUNCATE_MASK)
      for j in range(0, num_mantissa-1):
        mask = mask >> 1
        mask = mask | 0x80000000
      
      man_bits = bitwise_ops.bitwise_and(man_bits, mask)

      man_bits = bitwise_ops.left_shift(man_bits, diff_tensor)

      final_value = bitwise_ops.bitwise_or(sign_bit,exp_bits)
      final_value = bitwise_ops.bitwise_or(final_value, man_bits)

      zeros_tensor = tf.zeros_like(man_bits)
      final_value = tf.where(shift_to_zero, zeros_tensor, final_value)

      final_value = bitwise_ops.bitwise_and(final_value, mask)

      x = tf.bitcast(final_value, tf.float32)

      if skip_processing == False:
        if is_tensor:
          t_value[t_num] = tf.constant(x)
        else:
          t_value[t_num] = x
      else:
        t_value = x


    elif (round_mode == "nearest"):
      # FIXME: Need to apply tf.where changes to the custom_round_not_tensor - or merge functionality

      # TODO: Need to fix the problem with there is a round up but it causes a carry into
      # bit position 23 - need to adjust both exponent and mantissa
      # This code rounds by doing the following:
      # 1) truncate to the correct # of bits
      # 2) check the bit to the right of the last valid bit
      # 3) If bit to the right (half_one) is 1, then need to round up
      # 4) round up by creating the correct value in floating point (with correct exp and use implicit 1 in mantissa)
      # 5) then doing an addition.
      # 6) In order to do this with the tensors, first mask off that half_one bit
      # 7) Then shift to the right to the 0th bit position
      # 8) cast this bit to float using tf.cast
      # 9) multiply the round value with this half_one bit/float
      # 10) add result to truncated value 

      mask = int(NEAREST_PTR)
      #print("This is the original mask: ",hex(mask))
      for j in range(1, num_mantissa):
        mask = mask >> 1
        mask = mask | NEAREST_PTR

      #print("This is the new mask: ", hex(mask))
      convert_x = tf.bitcast(x, tf.int32)
      #print("mantissa convert_x: ", bitwise_ops.bitwise_and(convert_x, MAN_MASK))
      truncate_x = bitwise_ops.bitwise_and(convert_x, mask)
      #print("truncate_x: ", truncate_x)

      mask2 = int(NEAREST_PTR)
      #mask_low = int(LOWEST_PTR)

      for j in range(0, num_mantissa):
        mask2 = mask2 >> 1
        # mask will have a 1 at the bit to the right of the last mantissa bit

      #print("This is mask2: ", hex(mask2))
      # half_one stores the bit to the right of the last mantissa bit
      half_one = bitwise_ops.bitwise_and(convert_x, mask2)
      half_one = bitwise_ops.right_shift(half_one, reduce_num -1)

      # Create a tensor of Boolean values
      half_bool = tf.cast(half_one, tf.bool)
      #print("half_bool:", half_bool)

      mantissa_bits = bitwise_ops.bitwise_and(truncate_x, MAN_MASK)

      rest_bits = bitwise_ops.bitwise_and(convert_x, SIGN_EXP_MASK)

      # Need to create a float for the round up value
      # then add to the floating point value
      # floating point implicit 1, so mantissa = 0
      
      exp_bits = bitwise_ops.bitwise_and(convert_x, EXP_MASK)
      exp_bits = bitwise_ops.right_shift(exp_bits,23)

      # detect denormal values by looking for 1 in exponent
      zeros_tensor = tf.zeros_like(exp_bits)
      denormal = tf.math.equal(exp_bits, zeros_tensor)

      #print("exp_bits: ", exp_bits)

      #################################################################
      # NOTE: This is - num_mantissa to move the binary point to the 
      # correct location for the round value
      ones_tensor = tf.ones_like(exp_bits)
      exp_bits = exp_bits - (num_mantissa * ones_tensor)
      denormal_round = tf.math.less(exp_bits,ones_tensor)

      #print("exp_bits: ", exp_bits)
      exp_bits = bitwise_ops.left_shift(exp_bits,23)
      sign_bit = bitwise_ops.bitwise_and(convert_x,SIGN_MASK)

      # round_up is the round value to add
      round_up = bitwise_ops.bitwise_or(exp_bits, sign_bit)

      final_x = bitwise_ops.bitwise_or(mantissa_bits, rest_bits)

      #print("final_x: ", final_x)
      x = tf.bitcast(final_x, tf.float32)
      round_up = tf.where(denormal_round, zeros_tensor, round_up)
      y = tf.bitcast(round_up, tf.float32)
      
      final_value = tf.where(half_bool, x + y, x)

      # filter out denormal from adding 
      final_value = tf.where(denormal, x, final_value)

      if is_tensor:    
        t_value[t_num] = tf.constant(final_value)
      else:
        t_value[t_num] = final_value

      #print("This is the new tensor: ", t_value[t_num].numpy())

    elif (round_mode == "nearest_even"):
      # page 412 of Computer Arithmetic book

      # FIXME: Need to apply tf.where changes to the custom_round_not_tensor - or merge functionality

      # TODO: Need to fix the problem with there is a round up but it causes a carry into
      # bit position 23 - need to adjust both exponent and mantissa
      # This code rounds by doing the following:
      # 1) truncate to the correct # of bits
      # 2) check the bit to the right of the last valid bit
      # 3) If bit to the right (half_one) is 1, then need to round up
      # 4) round up by creating the correct value in floating point (with correct exp and use implicit 1 in mantissa)
      # 5) then doing an addition.
      # 6) In order to do this with the tensors, first mask off that half_one bit
      # 7) Then shift to the right to the 0th bit position
      # 8) cast this bit to float using tf.cast
      # 9) multiply the round value with this half_one bit/float
      # 10) add result to truncated value

      mask = int(NEAREST_PTR)
      #print("This is the original mask: ",hex(mask))
      for j in range(1, num_mantissa):
        mask = mask >> 1
        mask = mask | NEAREST_PTR

      #print("This is the new mask: ", hex(mask))
      convert_x = tf.bitcast(x, tf.int32)
      #print("mantissa convert_x: ", bitwise_ops.bitwise_and(convert_x, MAN_MASK))
      truncate_x = bitwise_ops.bitwise_and(convert_x, mask)
      #print("truncate_x: ", truncate_x)

      mask2 = int(NEAREST_PTR)
      #mask_low = int(LOWEST_PTR)

      for j in range(0, num_mantissa):
        mask2 = mask2 >> 1
        # mask will have a 1 at the bit to the right of the last mantissa bit

      #print("This is mask2: ", hex(mask2))
      # half_one stores the bit to the right of the last mantissa bit
      half_one = bitwise_ops.bitwise_and(convert_x, mask2)
      half_one = bitwise_ops.right_shift(half_one, reduce_num -1)

      # Create a tensor of Boolean values
      half_bool = tf.cast(half_one, tf.bool)
      #print("half_bool:", half_bool)

      mantissa_bits = bitwise_ops.bitwise_and(truncate_x, MAN_MASK)

      rest_bits = bitwise_ops.bitwise_and(convert_x, SIGN_EXP_MASK)

      # Need to create a float for the round up value
      # then add to the floating point value
      # floating point implicit 1, so mantissa = 0
      
      exp_bits = bitwise_ops.bitwise_and(convert_x, EXP_MASK)
      exp_bits = bitwise_ops.right_shift(exp_bits,23)

      # detect denormal values by looking for 1 in exponent
      zeroes_tensor = tf.zeros_like(exp_bits)
      denormal = tf.math.equal(exp_bits, zeroes_tensor)

      #print("exp_bits: ", exp_bits)

      #################################################################
      # NOTE: This is - num_mantissa to move the binary point to the 
      # correct location for the round value
      exp_bits = exp_bits - num_mantissa

      #print("exp_bits: ", exp_bits)
      exp_bits = bitwise_ops.left_shift(exp_bits,23)
      sign_bit = bitwise_ops.bitwise_and(convert_x,SIGN_MASK)

      # round_up is the round value to add
      round_up = bitwise_ops.bitwise_or(exp_bits, sign_bit)

      final_x = bitwise_ops.bitwise_or(mantissa_bits, rest_bits)

      #print("final_x: ", final_x)
      x = tf.bitcast(final_x, tf.float32)
      y = tf.bitcast(round_up, tf.float32)
      
      if (num_mantissa < 22):
        mask3 = 0x1
        for j in range(0,23 - num_mantissa - 2):
          mask3 = mask3 << 1
          mask3 = mask3 | 0x1
        guard_bits = bitwise_ops.bitwise_and(convert_x,mask3)
        not_zero = tf.math.greater(guard_bits,zeroes_tensor)
        equal_zero = tf.math.equal(guard_bits,zeroes_tensor)

        mask4 = 0x1
        for j in range(0,23 - num_mantissa):
          mask4 = mask4 << 1
        lsb = bitwise_ops.bitwise_and(convert_x,mask4)
        lsb_odd = tf.math.greater(lsb, zeroes_tensor)
        half_bool = tf.where(lsb_even,)

        round_equal = tf.math.logical_and(equal_zero,lsb_odd)

        half_bool = tf.math.logical_and(half_bool, not_zero)
        half_bool = tf.math.logical_or(half_bool, round_equal)
      else:
        pass

      # mask will have a 1 at the bit to the right of the last mantissa bit
      #print("x: ", x)
      #print("y: ", y)
      #print("x+y: ", x+y)
      final_value = tf.where(half_bool, x + y, x)

      # filter out denormal from adding 
      final_value = tf.where(denormal, x, final_value)

      if is_tensor:    
        t_value[t_num] = tf.constant(final_value)
      else:
        t_value[t_num] = final_value

      #print("This is the new tensor: ", t_value[t_num].numpy())

    elif (round_mode == "ibm_stochastic"):
     # This implementation is based on the IBM paper:
     # https://arxiv.org/pdf/1812.08011.pdf

      convert_x = tf.bitcast(x, tf.int32)

      # Need to take the floor
      mask = int(TRUNCATE_MASK)
      remain_mask = 0x1
      for j in range(0, num_mantissa):
        mask = mask >> 1
        mask = mask | 0x80000000
      
      for j in range(1,reduce_num):
        remain_mask = remain_mask << 1
        remain_mask = remain_mask | 0x1

      #print("remain_mask: ", remain_mask)
      truncate_value = bitwise_ops.bitwise_and(convert_x, mask)
      remainder_value = bitwise_ops.bitwise_and(convert_x, remain_mask)
      #print("remain_value: ", remainder_value)

      round_exponent = bitwise_ops.bitwise_and(convert_x,EXP_MASK)
      round_exponent = bitwise_ops.right_shift(round_exponent, 23)
      exp_save = round_exponent
      round_exponent = round_exponent - num_mantissa

      # check for denormal round value
      ones_tensor = tf.ones_like(round_exponent)
      denormal_round = tf.math.less(round_exponent, ones_tensor)
      zeros_tensor = tf.zeros_like(round_exponent)
      round_exponent = tf.where(denormal_round, zeros_tensor, round_exponent)

      round_exponent = bitwise_ops.left_shift(round_exponent,23)
      round_sign = bitwise_ops.bitwise_and(convert_x,SIGN_MASK)
      
      round_value = bitwise_ops.bitwise_or(round_exponent,round_sign)

      float_round = tf.bitcast(round_value, tf.float32)
      float_x = tf.bitcast(truncate_value, tf.float32)
      #print("Truncated float_x: ", float_x)

      added_value = float_x + float_round
      #print("Added value : ", added_value)

      # need to move MSB of remainder  to the implicit one position of bit 23
      # adjust the exponent based on how many bits moved over
      # use cast instead of bitcast

      ############################

      float_remainder = tf.cast(remainder_value, tf.float32)
      epsilon = (2**(reduce_num + 1))

      probability = float_remainder / epsilon
      #print("prob: ", probability)

      random_prob = tf.random.uniform(shape = x.shape, maxval = 1, dtype=tf.float32)
      #print("random_prob: ", random_prob)
      random_prob = random_prob + probability
      #print("random_prob + prob: ", random_prob)

      ones_tensor = tf.ones_like(random_prob)
      final_probability = tf.math.greater(random_prob, ones_tensor)
      #print("final_probability: ", final_probability)
      
      #final_probability = tf.cond((probability + random_prob) > 1.0,
      #                            lambda: True, lambda: False)

      final_value = tf.where(final_probability, added_value, float_x)

      exp_bits = bitwise_ops.bitwise_and(convert_x,EXP_MASK)
      # detect denormal values by looking for 1 in exponent
      zeroes_tensor = tf.zeros_like(exp_bits)
      denormal = tf.math.equal(exp_bits, zeroes_tensor)

      final_value = tf.where(denormal, float_x, added_value)

      x = final_value

      if is_tensor:          
        t_value[t_num] = tf.constant(x)
      else:
        t_value[t_num] = x


    elif (round_mode == "jamming"):
      # Jamming performs a bit-level OR of the least significant bit(LSB)
      # and 3 subsequent guard bits (GBs) and places the result in the LSB
      # Code uses the truncation code and modifies the LSB

      # Code to truncate the mantissa to the new number of bits
      mask = int(TRUNCATE_MASK)
      #print("This is the original mask: ",hex(mask))
      for j in range(0, num_mantissa):
        mask = mask >> 1
        mask = mask | 0x80000000

      # Need to cast float into integer format in order to apply bitwise operations
      # then convert back
      convert_x = tf.bitcast(x, tf.int32)
      truncate_x = bitwise_ops.bitwise_and(convert_x, mask)

      lsb_mask = LOWEST_PTR << reduce_num 
      if (reduce_num > 3):
        g_mask = GUARD_MASK << (reduce_num -3)
        g_right = GUARD_RIGHT << (reduce_num -3)
        g_mid = GUARD_MID << (reduce_num -3)
        g_left = GUARD_LEFT << (reduce_num -3)
      elif (reduce_num == 2):
        g_mask = 0x00000003
        g_right = 0x00000000
        g_mid = GUARD_MID 
        g_left = GUARD_LEFT
      elif (reduce_num == 1):
        g_mask = 0x00000001
        g_right = 0x00000000
        g_mid = 0x00000000
        g_left = GUARD_LEFT
        pass
      else:
        g_mask = 0
        g_right = 0x00000000
        g_mid = 0x00000000
        g_left = 0x00000000

      #print("lsb_mask, g_left, g_mid, g_right: ", lsb_mask, g_left, g_mid, g_right)

      lsb = bitwise_ops.bitwise_and(convert_x, lsb_mask)
      #guard_bits = bitwise_ops.bitwise_and(convert_x,g_mask)
      guard_l = bitwise_ops.bitwise_and(convert_x,g_left)
      guard_m = bitwise_ops.bitwise_and(convert_x,g_mid)
      guard_r = bitwise_ops.bitwise_and(convert_x,g_right)

      #print("guard_r: ", guard_r)
      #print("guard_m: ", guard_m)
      #print("guard_l: ", guard_l)
      
      # Shift guard bits into same bit position as LSB
      # To do bitwise or on all the bits and place in the same LSB position
      guard_l = bitwise_ops.left_shift(guard_l,1)
      guard_m = bitwise_ops.left_shift(guard_m,2)
      guard_r = bitwise_ops.left_shift(guard_r,3)

      #print("guard_r: ", guard_r)
      #print("guard_m: ", guard_m)
      #print("guard_l: ", guard_l)

      lsb = bitwise_ops.bitwise_or(lsb,guard_r)
      lsb = bitwise_ops.bitwise_or(lsb,guard_m)
      lsb = bitwise_ops.bitwise_or(lsb,guard_l)

      final_x = bitwise_ops.bitwise_or(truncate_x, lsb) 
      x = tf.bitcast(final_x, tf.float32)

      if is_tensor:          
        t_value[t_num] = tf.constant(x)
      else:
        t_value[t_num] = x
      #print("This is the new tensor: ", t_value[t_num].numpy())
      #print("#############################")   

      pass   
    elif (round_mode == "hw_stochastic"):
      pass
    elif (round_mode == "redundant"):
      pass
    elif (round_mode == "custom"):
      pass
    else:
      print("Round Mode Not Found!!!!!!!!!!!!!!!")
  
  return t_value

In [5]:
def fine_grain_custom_round(t_value, round_mode = "truncate", skip_processing = True, block_round_mode = "fine_grain_2_contiguous_exponent_reuse",
                            num_exp = 8, num_mantissa = 23, radix_exp = 2, block_size = 16, 
                            radix_mantissa = 2, is_tensor = True):
      
      #elif (round_mode == "exponent_reuse_new_fine_grain_dimension"):
 
      # Fine grain breakup of numbers to create blocks for exponent reuse
      # Do a study on varying the block_size
      #########################################
      #block_size = 128

  # radix_exp used to send tensor level max exp

  MAN_MASK = 0x007FFFFF 
  EXP_MASK = 0x7F800000
  SIGN_MASK = 0x80000000
  SIGN_EXP_MASK = 0xFF800000
  GUARD_MASK = 0x00000007
  GUARD_RIGHT = 0x00000001
  GUARD_MID = 0x00000002 
  GUARD_LEFT = 0x00000004

  # This mask is shifted to the right by reduce_num
  # Upper bits ORd with 1's
  TRUNCATE_MASK = 0xFF800000

  # This will point to the digit after the last digit to keep
  NEAREST_PTR = 0x00400000
  LOWEST_PTR = 0x00000001

  MAN_OVERFLOW = 0x00800000

  # Number of bits to reduce based on new number of mantissa bits
  reduce_num = 23 - num_mantissa

  exp_range = num_mantissa
  if (block_round_mode == "2_contiguous_exponent_reuse_nearest_exponent_only" or
      block_round_mode == "2_contiguous_exponent_reuse_nearest" or
      block_round_mode == "2_contiguous_exponent_reuse_jamming" or
      block_round_mode == "fine_grain_2_contiguous_exponent_reuse" or 
      block_round_mode == "2_contiguous_exponent_reuse_implicit" or
      block_round_mode == "ideal_2_contiguous_exponent_reuse" or
      "2" in block_round_mode):
    exp_range = num_mantissa * 2
  elif  (block_round_mode == "1_contiguous_exponent_reuse_nearest" or
          block_round_mode == "exponent_reuse"):
    exp_range = num_mantissa
  elif (block_round_mode == "3_contiguous_exponent_reuse" or
        block_round_mode == "3_contiguous_exponent_reuse_nearest" or
        block_round_mode == "3_contiguous_exponent_reuse_jamming" or
        block_round_mode == "3_contiguous_exponent_reuse_nearest_exponent_only" or
        block_round_mode == "3_contiguous_exponent_reuse_implicit" or
        "3" in block_round_mode):
    exp_range = num_mantissa * 3
  elif ("4" in block_round_mode):
    exp_range = num_mantissa * 4        

  # need to traverse the tensor
  # convert tensor to numpy array
  for t_num in range(len(t_value)):
    #inner_loop = 0
    if is_tensor:
      x = t_value[t_num].numpy()
    else:
      x = t_value[t_num]

    convert_x = tf.bitcast(x, tf.int32)
    exp_bits = bitwise_ops.bitwise_and(convert_x, EXP_MASK)

    exp_bits = bitwise_ops.right_shift(exp_bits, 23)
    tensor_max = tf.math.reduce_max(exp_bits)
    tensor_min = tf.math.reduce_min(exp_bits)
    tensor_diff = tensor_max - tensor_min

    #print("tensor level exp range: ", tensor_diff)
    
    if (tensor_diff > exp_range):
      smaller_blocks = True
    else:
      smaller_blocks = False
       
    # Forcing to smaller blocks!
    smaller_blocks = True
    #mean_for_printing = tf.math.reduce_mean(exp_bits)
    #print("max: ", max_for_printing, " mean: ", mean_for_printing, " min: ", min_for_printing)

    #print("#############################")
    #print("This is the original tensor: ", x)

    if (smaller_blocks == False):
      custom_round(x, skip_processing = True, round_mode = block_round_mode, 
                   num_mantissa=num_mantissa, is_tensor = False)
    else:
      # Breaking tensor into smaller blocks
      total_elements = tf.size(x).numpy()
      total_dimensions = x.ndim
      
      if (total_dimensions == 1) or (total_dimensions == 2 and x.shape[1] == 1):
        
        #block = x
        #custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
        #x = block
        
        start = 0
        stop = block_size
        while total_elements > block_size:
            total_elements = total_elements - block_size
            block = x[start:stop]
            #print("block: ", block)
            block = custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False, radix_exp = tensor_max)
            #print("after block: ", block)
            x[start:stop] = block
            start += block_size
            stop += block_size

        if (total_elements != 0): # round remaining values
            stop = start + total_elements
            block = x[start:stop]
            block = custom_round(block,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False, radix_exp = tensor_max)
            x[start:stop] = block
        
      elif (total_dimensions == 2) or (total_dimensions == 3 and x.shape[2] == 1):

        for i in range(x.shape[0]):
          #print("dim 2 inner loop: ", inner_loop)
          #inner_loop += 1

          ######
          total_elements = x.shape[1]
          start = 0
          stop = block_size

          

          while total_elements > block_size:
              total_elements = total_elements - block_size
              block = x[i, start:stop]
              #print("block: ", block)
              block = custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False, radix_exp = tensor_max)
              #print("after block: ", block)
              x[i, start:stop] = block
              start += block_size
              stop += block_size

          if (total_elements != 0): # round remaining values
              stop = start + total_elements
              block = x[i, start:stop]
              block = custom_round(block,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False, radix_exp = tensor_max)
              x[i, start:stop] = block
          ######
          '''
          row_x = x[i, :]
          #print("row_x: ", row_x)
          row_x = custom_round(row_x,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
          #print("after row_x: ", row_x)
          x[i, :] = row_x
          '''

      elif (total_dimensions == 3) or (total_dimensions == 4 and x.shape[3] == 1):
        for i in range(x.shape[0]):
          for j in range(x.shape[1]):
            #print("dim 3 inner loop: ", inner_loop)
            #inner_loop += 1

            ######
            total_elements = x.shape[2]
            start = 0
            stop = block_size
            while total_elements > block_size:
                total_elements = total_elements - block_size
                block = x[i, j, start:stop]
                #print("block: ", block)
                block = custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False, radix_exp = tensor_max)
                #print("after block: ", block)
                x[i, j, start:stop] = block
                start += block_size
                stop += block_size
                #if (total_dimensions == 4):
                #  print("block: ", block)

            if (total_elements != 0): # round remaining values
                stop = start + total_elements
                block = x[i, j, start:stop]
                block = custom_round(block,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False, radix_exp = tensor_max)
                x[i, j, start:stop] = block
            ######
            '''
            row_x = x[i,j, :]
            #print("row_x: ", row_x)
            row_x = custom_round(row_x,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
            #print("after row_x: ", row_x)
            x[i, j, :] = row_x
            '''

      elif (total_dimensions == 4):
        for i in range(x.shape[0]):
          for j in range(x.shape[1]):
            for k in range(x.shape[2]):
              #print("dim 4 inner loop: ", inner_loop)
              #inner_loop += 1

              ######
              total_elements = x.shape[3]
              start = 0
              stop = block_size
              while total_elements > block_size:
                  total_elements = total_elements - block_size
                  block = x[i, j, k, start:stop]
                  #print("block: ", block)
                  block = custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False, radix_exp = tensor_max)
                  #print("after block: ", block)
                  x[i, j, k, start:stop] = block
                  start += block_size
                  stop += block_size

              if (total_elements != 0): # round remaining values
                  stop = start + total_elements
                  block = x[i, j, k, start:stop]
                  block = custom_round(block,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False, radix_exp = tensor_max)
                  x[i, j, k, start:stop] = block
              ######
              '''
              row_x = x[i,j,k, :]
              #print("row_x: ", row_x)
              #print("x row: ", x[i,j,k, :])
              row_x = custom_round(row_x,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
              #print("after row_x: ", row_x)
              x[i, j, k, :] = row_x
              #print("after x row: ", x[i,j,k, :])
              '''
      else:
        print("Warning: tensor with dimension > 4!!!")

    if is_tensor:          
      t_value[t_num] = tf.constant(x)
    else:
      t_value[t_num] = x

  #print("#############################")
  #print("This is the new tensor: ", x)          
  return t_value  

In [6]:
def new_fine_grain_custom_round(t_value, round_mode = "truncate", skip_processing = True, block_round_mode = "fine_grain_2_contiguous_exponent_reuse",
                            num_exp = 8, num_mantissa = 23, radix_exp = 2, block_size = 16, 
                            radix_mantissa = 2, is_tensor = True):
      
      #elif (round_mode == "exponent_reuse_new_fine_grain_dimension"):
 
      # Fine grain breakup of numbers to create blocks for exponent reuse
      # Do a study on varying the block_size
      #########################################
      #block_size = 128

  MAN_MASK = 0x007FFFFF 
  EXP_MASK = 0x7F800000
  SIGN_MASK = 0x80000000
  SIGN_EXP_MASK = 0xFF800000
  GUARD_MASK = 0x00000007
  GUARD_RIGHT = 0x00000001
  GUARD_MID = 0x00000002 
  GUARD_LEFT = 0x00000004

  # This mask is shifted to the right by reduce_num
  # Upper bits ORd with 1's
  TRUNCATE_MASK = 0xFF800000

  # This will point to the digit after the last digit to keep
  NEAREST_PTR = 0x00400000
  LOWEST_PTR = 0x00000001

  MAN_OVERFLOW = 0x00800000

  # Number of bits to reduce based on new number of mantissa bits
  reduce_num = 23 - num_mantissa

  exp_range = num_mantissa
  if (block_round_mode == "2_contiguous_exponent_reuse_nearest_exponent_only" or
      block_round_mode == "2_contiguous_exponent_reuse_nearest" or
      block_round_mode == "2_contiguous_exponent_reuse_jamming" or
      block_round_mode == "fine_grain_2_contiguous_exponent_reuse" or 
      block_round_mode == "2_contiguous_exponent_reuse_implicit" or
      block_round_mode == "ideal_2_contiguous_exponent_reuse" or
      "2" in block_round_mode):
    exp_range = num_mantissa * 2
  elif  (block_round_mode == "1_contiguous_exponent_reuse_nearest" or
          block_round_mode == "exponent_reuse"):
    exp_range = num_mantissa
  elif (block_round_mode == "3_contiguous_exponent_reuse" or
        block_round_mode == "3_contiguous_exponent_reuse_nearest" or
        block_round_mode == "3_contiguous_exponent_reuse_jamming" or
        block_round_mode == "3_contiguous_exponent_reuse_nearest_exponent_only" or
        block_round_mode == "3_contiguous_exponent_reuse_implicit" or
        "3" in block_round_mode):
    exp_range = num_mantissa * 3
  elif ("4" in block_round_mode):
    exp_range = num_mantissa * 4        

  # need to traverse the tensor
  # convert tensor to numpy array
  for t_num in range(len(t_value)):
    #inner_loop = 0
    if is_tensor:
      x = t_value[t_num].numpy()
    else:
      x = t_value[t_num]

    convert_x = tf.bitcast(x, tf.int32)
    exp_bits = bitwise_ops.bitwise_and(convert_x, EXP_MASK)

    exp_bits = bitwise_ops.right_shift(exp_bits, 23)
    tensor_max = tf.math.reduce_max(exp_bits)
    tensor_min = tf.math.reduce_min(exp_bits)
    tensor_diff = tensor_max - tensor_min
    
    if (tensor_diff > exp_range):
      smaller_blocks = True
    else:
      smaller_blocks = False
       
    #mean_for_printing = tf.math.reduce_mean(exp_bits)
    #print("max: ", max_for_printing, " mean: ", mean_for_printing, " min: ", min_for_printing)

    #print("#############################")
    #print("This is the original tensor: ", x)

    if (smaller_blocks == False):
      custom_round(x, skip_processing = True, round_mode = block_round_mode, 
                   num_mantissa=num_mantissa, is_tensor = False)
    else:
      # Breaking tensor into smaller blocks
      total_elements = tf.size(x).numpy()
      total_dimensions = x.ndim
      
      if (total_dimensions == 1):
        
        #block = x
        #custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
        #x = block
        
        start = 0
        stop = block_size
        while total_elements > block_size:
            total_elements = total_elements - block_size
            block = x[start:stop]
            #print("block: ", block)
            block = custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
            #print("after block: ", block)
            x[start:stop] = block
            start += block_size
            stop += block_size

        if (total_elements != 0): # round remaining values
            stop = start + total_elements
            block = x[start:stop]
            block = custom_round(block,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
            x[start:stop] = block
        
      elif (total_dimensions == 2):

        # of last axis elements to take for each block if block_size > total elements
        num_axis_elements = block_size // x.shape[1]

        if num_axis_elements <= 1:
   
          for i in range(x.shape[0]):
            #print("dim 2 inner loop: ", inner_loop)
            #inner_loop += 1

            total_elements = x.shape[1]
            start = 0
            stop = block_size

            while total_elements > block_size:
                total_elements = total_elements - block_size
                block = x[i, start:stop]
                #print("block: ", block)
                block = custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
                #print("after block: ", block)
                x[i, start:stop] = block
                start += block_size
                stop += block_size

            if (total_elements != 0): # round remaining values
                stop = start + total_elements
                block = x[i, start:stop]
                block = custom_round(block,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
                x[i, start:stop] = block

        else:
          # Multiple last axis elements for each block

            total_elements = x.shape[0]
            start = 0
            stop = num_axis_elements
            block_size = num_axis_elements

            while total_elements > block_size:
                total_elements = total_elements - block_size
                block = x[start:stop, :]
                #print("block: ", block)
                block = custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
                #print("after block: ", block)
                x[start:stop, :] = block
                start += block_size
                stop += block_size

            if (total_elements != 0): # round remaining values
                stop = start + total_elements
                block = x[start:stop, :]
                block = custom_round(block,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
                x[start:stop,:] = block


      elif (total_dimensions == 3):

        num_axis_elements = block_size // x.shape[2]

        if num_axis_elements <= 1:

          for i in range(x.shape[0]):
            for j in range(x.shape[1]):
              #print("dim 3 inner loop: ", inner_loop)
              #inner_loop += 1

              ######
              total_elements = x.shape[2]
              start = 0
              stop = block_size
              while total_elements > block_size:
                  total_elements = total_elements - block_size
                  block = x[i, j, start:stop]
                  #print("block: ", block)
                  block = custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
                  #print("after block: ", block)
                  x[i, j, start:stop] = block
                  start += block_size
                  stop += block_size

              if (total_elements != 0): # round remaining values
                  stop = start + total_elements
                  block = x[i, j, start:stop]
                  block = custom_round(block,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
                  x[i, j, start:stop] = block
 
        else:
            num_axis_elements2 = num_axis_elements // x.shape[1]

            for i in range(x.shape[0]):
              total_elements = x.shape[1]
              start = 0
              stop = num_axis_elements2
              block_size = num_axis_elements2

              while total_elements > block_size:
                  total_elements = total_elements - block_size
                  block = x[i, start:stop, :]
                  #print("block: ", block)
                  block = custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
                  #print("after block: ", block)
                  x[i, start:stop, :] = block
                  start += block_size
                  stop += block_size

              if (total_elements != 0): # round remaining values
                  stop = start + total_elements
                  block = x[i, start:stop, :]
                  block = custom_round(block,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
                  x[i, start:stop, :] = block



      elif (total_dimensions == 4):

        num_axis_elements = block_size // x.shape[3]

        if num_axis_elements <= 1:

          for i in range(x.shape[0]):
            for j in range(x.shape[1]):
              for k in range(x.shape[2]):
                #print("dim 4 inner loop: ", inner_loop)
                #inner_loop += 1

                ######
                total_elements = x.shape[3]
                start = 0
                stop = block_size
                while total_elements > block_size:
                    total_elements = total_elements - block_size
                    block = x[i, j, k, start:stop]
                    #print("block: ", block)
                    block = custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
                    #print("after block: ", block)
                    x[i, j, k, start:stop] = block
                    start += block_size
                    stop += block_size

                if (total_elements != 0): # round remaining values
                    stop = start + total_elements
                    block = x[i, j, k, start:stop]
                    block = custom_round(block,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
                    x[i, j, k, start:stop] = block

        else:

          num_axis_elements2 = num_axis_elements // x.shape[2]

          for i in range(x.shape[0]):
            for j in range(x.shape[1]):

              total_elements = x.shape[2]
              start = 0
              stop = num_axis_elements2
              block_size = num_axis_elements2              

              while total_elements > block_size:
                  total_elements = total_elements - block_size
                  block = x[i, j, start:stop, :]
                  #print("block: ", block)
                  block = custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
                  #print("after block: ", block)
                  x[i, j, start:stop, :] = block
                  start += block_size
                  stop += block_size

              if (total_elements != 0): # round remaining values
                  stop = start + total_elements
                  block = x[i, j, start:stop, :]
                  block = custom_round(block,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
                  x[i, j, start:stop, :] = block          


    if is_tensor:          
      t_value[t_num] = tf.constant(x)
    else:
      t_value[t_num] = x

  #print("#############################")
  #print("This is the new tensor: ", x)          
  return t_value  

In [7]:
def layer_outputs_custom_round(t_value, round_mode = "truncate", skip_processing = True, block_round_mode = "fine_grain_2_contiguous_exponent_reuse",
                              num_exp = 8, num_mantissa = 23, radix_exp = 2, block_size = 16, 
                              radix_mantissa = 2, is_tensor = True):
      
      #elif (round_mode == "exponent_reuse_new_fine_grain_dimension"):
 
      # Fine grain breakup of numbers to create blocks for exponent reuse
      # Do a study on varying the block_size
      #########################################
      #block_size = 128

  MAN_MASK = 0x007FFFFF 
  EXP_MASK = 0x7F800000
  SIGN_MASK = 0x80000000
  SIGN_EXP_MASK = 0xFF800000
  GUARD_MASK = 0x00000007
  GUARD_RIGHT = 0x00000001
  GUARD_MID = 0x00000002 
  GUARD_LEFT = 0x00000004

  # This mask is shifted to the right by reduce_num
  # Upper bits ORd with 1's
  TRUNCATE_MASK = 0xFF800000

  # This will point to the digit after the last digit to keep
  NEAREST_PTR = 0x00400000
  LOWEST_PTR = 0x00000001

  MAN_OVERFLOW = 0x00800000

  # Number of bits to reduce based on new number of mantissa bits
  reduce_num = 23 - num_mantissa

  exp_range = num_mantissa
  if (block_round_mode == "2_contiguous_exponent_reuse_nearest_exponent_only" or
      block_round_mode == "2_contiguous_exponent_reuse_nearest" or
      block_round_mode == "2_contiguous_exponent_reuse_jamming" or
      block_round_mode == "fine_grain_2_contiguous_exponent_reuse" or 
      block_round_mode == "2_contiguous_exponent_reuse_implicit" or
      block_round_mode == "ideal_2_contiguous_exponent_reuse" or
      "2" in block_round_mode):
    exp_range = num_mantissa * 2
  elif  (block_round_mode == "1_contiguous_exponent_reuse_nearest" or
          block_round_mode == "exponent_reuse"):
    exp_range = num_mantissa
  elif (block_round_mode == "3_contiguous_exponent_reuse" or
        block_round_mode == "3_contiguous_exponent_reuse_nearest" or
        block_round_mode == "3_contiguous_exponent_reuse_jamming" or
        block_round_mode == "3_contiguous_exponent_reuse_nearest_exponent_only" or
        block_round_mode == "3_contiguous_exponent_reuse_implicit" or
        "3" in block_round_mode):
    exp_range = num_mantissa * 3
  elif ("4" in block_round_mode):
    exp_range = num_mantissa * 4        

  # need to traverse the tensor
  # convert tensor to numpy array
  '''
  for t_num in range(len(t_value)):
    #inner_loop = 0
    if is_tensor:
      x = t_value[t_num].numpy()
    else:
      x = t_value[t_num]
  '''

  x = t_value.numpy()

  convert_x = tf.bitcast(x, tf.int32)
  exp_bits = bitwise_ops.bitwise_and(convert_x, EXP_MASK)

  exp_bits = bitwise_ops.right_shift(exp_bits, 23)
  tensor_max = tf.math.reduce_max(exp_bits)
  tensor_min = tf.math.reduce_min(exp_bits)
  tensor_diff = tensor_max - tensor_min
  
  if (tensor_diff > exp_range):
    smaller_blocks = True
  else:
    smaller_blocks = False
      
  #mean_for_printing = tf.math.reduce_mean(exp_bits)
  #print("max: ", max_for_printing, " mean: ", mean_for_printing, " min: ", min_for_printing)

  #print("#############################")
  #print("This is the original tensor: ", x)

  smaller_blocks = True
  if (smaller_blocks == False):
    custom_round(x, skip_processing = True, round_mode = block_round_mode, 
                  num_mantissa=num_mantissa, is_tensor = False)
  else:
    # Breaking tensor into smaller blocks
    total_elements = tf.size(x).numpy()
    total_dimensions = x.ndim
    
    if (total_dimensions == 1):
      
      #block = x
      #custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
      #x = block
      
      start = 0
      stop = block_size
      while total_elements > block_size:
          total_elements = total_elements - block_size
          block = x[start:stop]
          #print("block: ", block)
          block = custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
          #print("after block: ", block)
          x[start:stop] = block
          start += block_size
          stop += block_size

      if (total_elements != 0): # round remaining values
          stop = start + total_elements
          block = x[start:stop]
          block = custom_round(block,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
          x[start:stop] = block
      
    elif (total_dimensions == 2):

      for i in range(x.shape[0]):
        #print("dim 2 inner loop: ", inner_loop)
        #inner_loop += 1

        ######
        total_elements = x.shape[1]
        start = 0
        stop = block_size
        while total_elements > block_size:
            total_elements = total_elements - block_size
            block = x[i, start:stop]
            #print("block: ", block)
            block = custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
            #print("after block: ", block)
            x[i, start:stop] = block
            start += block_size
            stop += block_size

        if (total_elements != 0): # round remaining values
            stop = start + total_elements
            block = x[i, start:stop]
            block = custom_round(block,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
            x[i, start:stop] = block
        ######
        '''
        row_x = x[i, :]
        #print("row_x: ", row_x)
        row_x = custom_round(row_x,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
        #print("after row_x: ", row_x)
        x[i, :] = row_x
        '''

    elif (total_dimensions == 3):
      for i in range(x.shape[0]):
        for j in range(x.shape[1]):
          #print("dim 3 inner loop: ", inner_loop)
          #inner_loop += 1

          ######
          total_elements = x.shape[2]
          start = 0
          stop = block_size
          while total_elements > block_size:
              total_elements = total_elements - block_size
              block = x[i, j, start:stop]
              #print("block: ", block)
              block = custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
              #print("after block: ", block)
              x[i, j, start:stop] = block
              start += block_size
              stop += block_size

          if (total_elements != 0): # round remaining values
              stop = start + total_elements
              block = x[i, j, start:stop]
              block = custom_round(block,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
              x[i, j, start:stop] = block
          ######
          '''
          row_x = x[i,j, :]
          #print("row_x: ", row_x)
          row_x = custom_round(row_x,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
          #print("after row_x: ", row_x)
          x[i, j, :] = row_x
          '''

    elif (total_dimensions == 4):
      for i in range(x.shape[0]):
        for j in range(x.shape[1]):
          for k in range(x.shape[2]):
            #print("dim 4 inner loop: ", inner_loop)
            #inner_loop += 1

            ######
            total_elements = x.shape[3]
            start = 0
            stop = block_size
            while total_elements > block_size:
                total_elements = total_elements - block_size
                block = x[i, j, k, start:stop]
                #print("block: ", block)
                block = custom_round(block, skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
                #print("after block: ", block)
                x[i, j, k, start:stop] = block
                start += block_size
                stop += block_size

            if (total_elements != 0): # round remaining values
                stop = start + total_elements
                block = x[i, j, k, start:stop]
                block = custom_round(block,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
                x[i, j, k, start:stop] = block
            ######
            '''
            row_x = x[i,j,k, :]
            #print("row_x: ", row_x)
            #print("x row: ", x[i,j,k, :])
            row_x = custom_round(row_x,  skip_processing = skip_processing, round_mode = block_round_mode, num_mantissa=num_mantissa, is_tensor = False)
            #print("after row_x: ", row_x)
            x[i, j, k, :] = row_x
            #print("after x row: ", x[i,j,k, :])
            '''
  '''  
  if is_tensor:          
    t_value[t_num] = tf.constant(x)
  else:
    t_value[t_num] = x
  '''
  
  t_value = tf.constant(x)

#print("#############################")
#print("This is the new tensor: ", x)          
  return t_value  

In [8]:
# Verification code for custom rounding
testing_rounding = False

if (testing_rounding): 
  
  test_tensor = tf.constant([ 
    [[60, 1.5, 49.75, 0.25, 64],
    [5.25, 6.25, 7.25, 8.25, 8.45]],
    [[0.32, 0.22, 0.12, 0.52, 0.92],
    [0.52, 0.62, 0.72, 0.82, 0.2]],
    [[20.2, 21.2, 22.2, 23.2, 24.2],
    [25.2, 26.2, 27.2, 28.2, 29.2]],])

  test_tensor2 = tf.constant([
    [[40, 1.5, 49.75, 0.25, 38],
    [5.5, 6.5, 7.5, 8.5, 8.9]],
    [[10.2, 11.2, 12.2, 13.2, 14.2],
    [15.2, 16.2, 17.2, 18.2, 19.2]],
    [[20.2, 21.2, 22.2, 23.2, 24.2],
    [25.2, 26.2, 27.2, 28.2, 29.2]],])
  
  test_tensor3 = tf.constant([
    [[40, 1.5, 49.75, 0.25, 38],
    [5.5, 6.5, 7.5, 8.5, 8.9]],
    [[1.27, -2.25, 4, 6.6, 8],
    [15.2, 16.2, 17.2, 18.2, 19.2]],
    [[20.2, 21.2, 22.2, 23.2, 24.2],
    [25.2, 26.2, 27.2, 28.2, 29.2]],])


  tensor_list = []
  tensor_list.append(test_tensor)
  tensor_list.append(test_tensor2)
  tensor_list.append(test_tensor3)

  print("nearest")
  print("Original Tensor: ", tensor_list)
  #custom_round(tensor_list, round_mode = "nearest", num_mantissa = 2)
  # fine_grain_custom_round(tensor_list, skip_processing = True,  
  #                         round_mode = "exponent_reuse_new_fine_grain_dimension", num_mantissa = 3, 
  #                         block_size = 5, block_round_mode = "afp_sector_update")
  # print("Rounded Tensor: ", tensor_list)

  fine_grain_custom_round(tensor_list, skip_processing = True,  
                          round_mode = "exponent_reuse_new_fine_grain_dimension", num_mantissa = 3, 
                          block_size = 5, block_round_mode = "afp_sector_update")
  print("Rounded Tensor: ", tensor_list)




In [9]:
# Verification code for custom rounding
testing_rounding = False

if (testing_rounding): 
  
  test_tensor = tf.constant([ 
    [[60, 1.5, 49.75, 0.25, 64],
    [5.25, 6.25, 7.25, 8.25, 8.45]],
    [[0.32, 0.22, 0.12, 0.52, 0.92],
    [0.52, 0.62, 0.72, 0.82, 0.2]],
    [[20.2, 21.2, 22.2, 23.2, 24.2],
    [25.2, 26.2, 27.2, 28.2, 29.2]],])

  test_tensor2 = tf.constant([
    [[40, 1.5, 49.75, 0.25, 38],
    [5.5, 6.5, 7.5, 8.5, 8.9]],
    [[10.2, 11.2, 12.2, 13.2, 14.2],
    [15.2, 16.2, 17.2, 18.2, 19.2]],
    [[20.2, 21.2, 22.2, 23.2, 24.2],
    [25.2, 26.2, 27.2, 28.2, 29.2]],])
  
  test_tensor3 = tf.constant([
    [[40, 1.5, 49.75, 0.25, 38],
    [5.5, 6.5, 7.5, 8.5, 8.9]],
    [[1.27, -2.25, 4, 6.6, 8],
    [15.2, 16.2, 17.2, 18.2, 19.2]],
    [[20.2, 21.2, 22.2, 23.2, 24.2],
    [25.2, 26.2, 27.2, 28.2, 29.2]],])


  tensor_list = []
  tensor_list.append(test_tensor)
  tensor_list.append(test_tensor2)
  tensor_list.append(test_tensor3)

  print("nearest")
  print("Original Tensor: ", tensor_list)
  #custom_round(tensor_list, round_mode = "nearest", num_mantissa = 2)
  # fine_grain_custom_round(tensor_list, skip_processing = True,  
  #                         round_mode = "exponent_reuse_new_fine_grain_dimension", num_mantissa = 3, 
  #                         block_size = 5, block_round_mode = "afp_sector_update")
  # print("Rounded Tensor: ", tensor_list)

  fine_grain_custom_round(tensor_list, skip_processing = True,  
                          round_mode = "exponent_reuse_new_fine_grain_dimension", num_mantissa = 3, 
                          block_size = 5, block_round_mode = "afp_sector_update_jamming_1bit")
  print("Rounded Tensor: ", tensor_list)




In [10]:
# Verification code for custom rounding
# Need to debug jamming rounding - should not be rounding up.

testing_rounding = True

if (testing_rounding): 
  
  test_tensor = tf.constant([ 
    [[0, 1.5, 49.75, 0.25, 64],
    [5.63, 6.25, 7.25, 8.25, 8.45]],
    [[0.32, 0.22, 0.12, 0.52, 0.92],
    [0.52, 0.62, 0.72, 0.82, 0.2]],
    [[20.2, 21.2, 22.2, 23.2, 24.2],
    [25.2, 26.2, 27.2, 28.2, 29.2]],])

  test_tensor2 = tf.constant([
    [[40.25, 1.5, 49.75, 0.125, 38],
    [5.5, 6.5, 7.5, 8.5, 8.9]],
    [[10.2, 11.2, 12.2, 13.2, 14.2],
    [15.2, 16.2, 17.2, 18.2, 19.2]],
    [[20.2, 21.2, 22.2, 23.2, 24.2],
    [25.2, 26.2, 27.2, 28.2, 29.2]],])
  
  test_tensor3 = tf.constant([
    [[40, 1.5, 49.75, 0.25, 38],
    [5.5, 6.5, 7.5, 8.5, 8.9]],
    [[1.27, -2.25, 4, 6.6, 8],
    [15.2, 16.2, 17.2, 18.2, 19.2]],
    [[20.2, 21.2, 22.2, 23.2, 24.2],
    [25.2, 26.2, 27.2, 28.2, 29.2]],])


  tensor_list = []
  tensor_list.append(test_tensor)
  tensor_list.append(test_tensor2)
  tensor_list.append(test_tensor3)

  print("nearest")
  print("Original Tensor: ", tensor_list)
  #custom_round(tensor_list, round_mode = "nearest", num_mantissa = 2)
  # fine_grain_custom_round(tensor_list, skip_processing = True,  
  #                         round_mode = "exponent_reuse_new_fine_grain_dimension", num_mantissa = 3, 
  #                         block_size = 5, block_round_mode = "afp_sector_update")
  # print("Rounded Tensor: ", tensor_list)

  fine_grain_custom_round(tensor_list, skip_processing = True,  
                          round_mode = "exponent_reuse_new_fine_grain_dimension", num_mantissa = 3, 
                          block_size = 5, block_round_mode = "afp_sector_update_jamming_zero_new")
  print("Rounded Tensor: ", tensor_list)




nearest
Original Tensor:  [<tf.Tensor: shape=(3, 2, 5), dtype=float32, numpy=
array([[[ 0.  ,  1.5 , 49.75,  0.25, 64.  ],
        [ 5.63,  6.25,  7.25,  8.25,  8.45]],

       [[ 0.32,  0.22,  0.12,  0.52,  0.92],
        [ 0.52,  0.62,  0.72,  0.82,  0.2 ]],

       [[20.2 , 21.2 , 22.2 , 23.2 , 24.2 ],
        [25.2 , 26.2 , 27.2 , 28.2 , 29.2 ]]], dtype=float32)>, <tf.Tensor: shape=(3, 2, 5), dtype=float32, numpy=
array([[[40.25 ,  1.5  , 49.75 ,  0.125, 38.   ],
        [ 5.5  ,  6.5  ,  7.5  ,  8.5  ,  8.9  ]],

       [[10.2  , 11.2  , 12.2  , 13.2  , 14.2  ],
        [15.2  , 16.2  , 17.2  , 18.2  , 19.2  ]],

       [[20.2  , 21.2  , 22.2  , 23.2  , 24.2  ],
        [25.2  , 26.2  , 27.2  , 28.2  , 29.2  ]]], dtype=float32)>, <tf.Tensor: shape=(3, 2, 5), dtype=float32, numpy=
array([[[40.  ,  1.5 , 49.75,  0.25, 38.  ],
        [ 5.5 ,  6.5 ,  7.5 ,  8.5 ,  8.9 ]],

       [[ 1.27, -2.25,  4.  ,  6.6 ,  8.  ],
        [15.2 , 16.2 , 17.2 , 18.2 , 19.2 ]],

       [[20.2 , 21.2

In [11]:
# sum(|rounded - original|)/ total_num -> average error
# sum(|rounded - original / original|) / total_sum -> average percentage error
from math import log

def average_error(p, q):
  ae_average = 0
  ae_total = 0
  total_num = 0

  for i, elem in enumerate(p):
    total_num += tf.size(elem).numpy()
    # zero_tensor = tf.zeros_like(elem)
    # boolean_tensor = tf.math.equal(elem, zero_tensor)
    x = tf.math.subtract(elem, q[i])
    x = tf.math.abs(x)
    x = tf.math.reduce_sum(x)
    ae_total += x.numpy()
  if total_num == 0:
    pass
  else:
    ae_average = ae_total / total_num

  return ae_average, ae_total, total_num

In [12]:
from math import log
# sum(|rounded - original / original|) / total_sum -> average percentage error
def percent_average_error(p, q):
  ae_average = 0
  ae_total = 0
  total_num = 0

  for i, elem in enumerate(p):
    total_num += tf.size(elem).numpy()
    # zero_tensor = tf.zeros_like(elem)
    # boolean_tensor = tf.math.equal(elem, zero_tensor)
    x = tf.math.subtract(elem, q[i])
    x = tf.math.divide(x, q[i])
    x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
    x = tf.math.abs(x)
    x = tf.math.reduce_sum(x)
    ae_total += x.numpy()
  if total_num == 0:
    pass
  else:
    ae_average = ae_total / total_num

  return ae_average, ae_total, total_num

In [13]:


# This model uses the 2 different modes of AFP to deal with Batch Norm Layers

########
#2ND RUN
########
import tensorflow as tf
import cv2
import numpy
###############
import copy

#ds = ds.shuffle(1024).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
round_types = ["exponent_reuse_new_fine_grain_dimension"]
block_round_types = ["afp_sector_update_jamming_zero_new"]
num_man = 23
test_images = []
labels = []
acc = tf.keras.metrics.CategoricalAccuracy()

block_size_list = [16]

min_param_size = 10000

first_layers = []
second_layers = []

skip_layers = True

#sample = ds.take(100)

round_mode = "exponent_reuse_new_fine_grain_dimension"

for block_round_mode in block_round_types:

  for i in range(len(block_size_list)):

    block_size = block_size_list[i]
    if (round_mode != "exponent_reuse_new_fine_grain_dimension" and i > 0):
      continue

    for num_mantissa in range(4, 2, -1):
 

      acc.reset_states()
      model = EfficientNetB0(weights='imagenet')

      # 35 out of 49
      ####################
      # Initializing accuracy metric for per layer
      # 16 correct out of 20
      # data = [number_correct, total_predictions]
      # initialize to data = [0.0, 0.0]
      data = [0.0, 0.0]
      image_num = int(data[1])
      data[0] = tf.cast(data[0], tf.float32)
      data[1] = tf.cast(data[1], tf.float32)
      acc.set_weights(data)
      #print("Image: ", image_num, " Accuracy: ", (acc.result().numpy()*100))
      ds = ds.skip(image_num)
      ####################      

      rounded_layers = []
      for layer_elem in model.layers:
        #print("layer name: ", layer_elem.name)
        # if layer_elem.count_params() < min_param_size: 
        #   continue 
        # rounded_layers.append(layer_elem.name)

        if ((skip_layers == False or "conv" in layer_elem.name) and (skip_layers == False or "bn" not in layer_elem.name)):
          #print("layer_num: ", layer_num)
          layer_weights = layer_elem.get_weights()
          #print("layer shape: ", tf.shape(layer_weights))
          if (round_mode == "exponent_reuse_new_fine_grain_dimension"):
            fine_grain_custom_round(layer_weights, round_mode = round_mode, skip_processing = True, 
                                        block_round_mode = block_round_mode, block_size = block_size, 
                                        num_mantissa=num_mantissa, is_tensor = False)
          else:
            custom_round(layer_weights, round_mode, num_mantissa=num_mantissa, is_tensor = False)
          layer_elem.set_weights(layer_weights)
        '''
        elif (skip_layers == True and "bn" in layer_elem.name):
          
          layer_weights = layer_elem.get_weights()

          if (round_mode == "exponent_reuse_new_fine_grain_dimension"):
            fine_grain_custom_round(layer_weights, round_mode = round_mode, skip_processing = True, 
                                    block_round_mode = "afp_optimal_range_msfp_nearest_v2", block_size = block_size, 
                                    num_mantissa=num_mantissa, is_tensor = False)
          else:
            custom_round(layer_weights, round_mode = "afp_optimal_range_msfp_nearest_v2", num_mantissa=num_mantissa, is_tensor = False)
        '''  

      # 100 samples for faster testing
  #    for example in ds.take(100):
  #    for example in ds:
      #image_num = 0
      #image_num = 0
      for example in ds:
        image_num += 1

        image_elem, label_elem = example["image"], example["label"]
        test_image = image.img_to_array(image_elem)

        # Resize

        height, width, _ = test_image.shape
        new_height = height * 256 // min(test_image.shape[:2])
        new_width = width * 256 // min(test_image.shape[:2])
        test_image = cv2.resize(test_image, (new_width, new_height), interpolation=cv2.INTER_CUBIC)

        # Crop
        height, width, _ = test_image.shape
        startx = width//2 - (224//2)
        starty = height//2 - (224//2)
        test_image = test_image[starty:starty+224,startx:startx+224]
        assert test_image.shape[0] == 224 and test_image.shape[1] == 224, (test_image.shape, height, width)
       
        test_image = np.expand_dims(test_image, axis=0)
        test_image = preprocess_input(test_image)

        accumulate_total = 0
        accumulate_num = 0

        paccumulate_total = 0
        paccumulate_num = 0         

        output_dict = {}
        first = True
        for layer in model.layers:
          if first: #for the first layer case
            x = layer(test_image)
            prev_x = copy.deepcopy(x)  

            # Apply rounding on x
            '''
            if layer.name in rounded_layers:
              if ((skip_layers == False or "conv" in layer.name) and (skip_layers == False or "bn" not in layer.name)):
                if (round_mode == "exponent_reuse_new_fine_grain_dimension"):
                  #print(x)
                  x = layer_outputs_custom_round(x, round_mode = round_mode, skip_processing = True, 
                                          block_round_mode = block_round_mode, block_size = block_size, 
                                          num_mantissa=num_mantissa, is_tensor = False)
                else:
                  custom_round(x, round_mode, num_mantissa=num_mantissa, is_tensor = False)
            '''
            output_dict[layer.name] = x
            first = False
          else: #for single input cases
            if not isinstance(layer.input, list):
              input_name = layer.input.name.split('/')[0]
              input = output_dict.get(input_name)
              x = layer(input)
              prev_x = copy.deepcopy(x)  
              # Apply rounding on x

              '''
              if layer.name in rounded_layers:
                if ((skip_layers == False or "conv" in layer.name) and (skip_layers == False or "bn" not in layer.name)):
                  if (round_mode == "exponent_reuse_new_fine_grain_dimension"):
                    #print(x)
                    x = layer_outputs_custom_round(x, round_mode = round_mode, skip_processing = True, 
                                            block_round_mode = block_round_mode, block_size = block_size, 
                                            num_mantissa=num_mantissa, is_tensor = False)
                  else:
                    custom_round(x, round_mode, num_mantissa=num_mantissa, is_tensor = False)
              '''

              layer_name = layer.name.split('/')[0]
              output_dict[layer_name] = x
            else: #for two input cases
              inputs = []
              for i, input_layer in enumerate(layer.input):
                input_name = layer.input[i].name.split('/')[0]
                input = output_dict.get(input_name)
                inputs.append(input)
              x = layer(inputs)
              prev_x = copy.deepcopy(x)  

              # Apply rounding on x
              '''
              if layer.name in rounded_layers:
                if ((skip_layers == False or "conv" in layer.name) and (skip_layers == False or "bn" not in layer.name)):
                  if (round_mode == "exponent_reuse_new_fine_grain_dimension"):
                    #print(x)
                    x = layer_outputs_custom_round(x, round_mode = round_mode, skip_processing = True, 
                                            block_round_mode = block_round_mode, block_size = block_size, 
                                            num_mantissa=num_mantissa, is_tensor = False)
                  else:
                    custom_round(x, round_mode, num_mantissa=num_mantissa, is_tensor = False)
              '''

              layer_name = layer.name.split('/')[0]
              output_dict[layer_name] = x

          #avr_error, num, total = average_error(x, prev_x)
          #pavr_error, pnum, ptotal = percent_average_error(x, prev_x)

          #accumulate_num += num
          #accumulate_total += total
          #paccumulate_num += pnum
          #paccumulate_total += ptotal 

        # if accumulate_num == 0:
        #   avg_total = 0
        # else:
        #   avg_total = accumulate_num / accumulate_total
        # if paccumulate_num == 0:
        #   pavg_total = 0
        # else:
        #   pavg_total = paccumulate_num / paccumulate_total 

        preds = x[0]
        label = tf.one_hot(label_elem, 1000)
        acc.update_state(label, preds)
        # per layer update
        raw_data = acc.get_weights()
        if (image_num % 1000 == 0):
          print("Image: ", image_num, " Accuracy: ", (acc.result().numpy()*100), "Raw Data: ", raw_data) #, "Error: ", avg_total, "Percent_Error: ", pavg_total)
 
      if (round_mode == "exponent_reuse_new_fine_grain_dimension"):
        print(
          f'DMG Round_mode: {block_round_mode}, Block_size: {block_size}, Num_mantissa: {num_mantissa}, '
          f'Accuracy: {(acc.result().numpy()*100)} '
        )
      else:
        print(
              f'DMG Round_mode: {round_mode}, Num_mantissa: {num_mantissa}, '
              f'Accuracy: {(acc.result().numpy()*100)} '
            )
  # decode the results into a list of tuples (class, description, probability)
  # (one such list for each sample in the batch)
# print(round_mode + " Num Mantissa: ", num_man, ' Predicted:', decode_predictions(preds, top=3)[0])

# Adding code to calculate Top-1 accuracy:
# Info from here: https://github.com/calebrob6/imagenet_validation/blob/master/2.%20Benchmark%20Keras%20pretrained%20models%20on%20ImageNet.ipynb

21839872/21834768 [==============================] - 0s 0us/step
Image:  1000  Accuracy:  50.599998235702515 Raw Data:  [506.0, 1000.0]
Image:  2000  Accuracy:  50.999999046325684 Raw Data:  [1020.0, 2000.0]
Image:  3000  Accuracy:  51.30000114440918 Raw Data:  [1539.0, 3000.0]
Image:  4000  Accuracy:  51.72500014305115 Raw Data:  [2069.0, 4000.0]
Image:  5000  Accuracy:  51.38000249862671 Raw Data:  [2569.0, 5000.0]
Image:  6000  Accuracy:  51.249998807907104 Raw Data:  [3075.0, 6000.0]
Image:  7000  Accuracy:  51.61428451538086 Raw Data:  [3613.0, 7000.0]
Image:  8000  Accuracy:  51.73749923706055 Raw Data:  [4139.0, 8000.0]
Image:  9000  Accuracy:  51.92221999168396 Raw Data:  [4673.0, 9000.0]
Image:  10000  Accuracy:  51.70999765396118 Raw Data:  [5171.0, 10000.0]
DMG Round_mode: afp_sector_update_jamming_zero_new, Block_size: 16, Num_mantissa: 4, Accuracy: 51.70999765396118 
Image:  1000  Accuracy:  6.400000303983688 Raw Data:  [64.0, 1000.0]
Image:  2000  Accuracy:  6.80000036954


DMG Round_mode: afp_sector_update_jamming_zero_new, Block_size: 16, Num_mantissa: 6, Accuracy: 62.199997901916504 

DMG Round_mode: afp_sector_update_jamming_zero_new, Block_size: 16, Num_mantissa: 5, Accuracy: 61.729997396469116 